<a href="https://colab.research.google.com/github/Danae-DB/danaediazbellenger/blob/main/proyectoIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto BALVIA

Importamos librerias iniciales a trabajar para subir pdf's a trabajar

In [1]:
!pip install -q pymupdf pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 28.3 MB/s eta 0:00:00


esto convierte todo el pdf a imagenes, esta probado y corrroborado en databricks, pero acá no es muy útil

In [ ]:
# @title
import fitz
from PIL import Image
import os


DPI = 200

os.makedirs(folder_img, exist_ok=True)

for archivo in os.listdir(folder_pdf):
    if not archivo.lower().endswith(".pdf"):
        continue

    pdf_path = os.path.join(folder_pdf, archivo)
    with fitz.open(pdf_path) as pdf_doc:
        n_pages = pdf_doc.page_count
        print(f"Procesando {archivo}, páginas: {n_pages}")

        nombre_pdf = os.path.splitext(archivo)[0]
        subfolder = os.path.join(folder_img, nombre_pdf)
        os.makedirs(subfolder, exist_ok=True)

        # Convertir DPI a escala (72 DPI base interna de PDF)
        zoom = DPI / 72.0
        mat = fitz.Matrix(zoom, zoom)

        for i in range(n_pages):
            page = pdf_doc.load_page(i)
            pix = page.get_pixmap(matrix=mat, alpha=False)  # alpha=False → RGB

            # Guardamos con ceros a la izquierda para orden estable
            nombre_imagen = os.path.join(subfolder, f"page_{i+1:04d}.png")

            # Convertir a PIL y guardar (opcional; pix.save también sirve)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            img.save(nombre_imagen, format="PNG", optimize=True)

        print(f"Imágenes del PDF '{archivo}' guardadas en {subfolder}")

#Aquí parte el código del Diplomado IA

# EasyOCR

instalo OCR

In [2]:
# @title
!pip install -q easyocr==1.7.1 opencv-python-headless==4.10.0.84 pillow==10.4.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 17.3 MB/s eta 0:00:00


importo librerias de ocr

### Intento con pdf finales

PyMuPDF (fitz) para leer PDFs y extraer tablas, Pillow para imágenes, pandas para la tabla final.
Opcional: Tesseract + pytesseract por si alguna página viene escaneada sin capa de texto (OCR).

In [3]:
# Parte 1: dependencias
!pip -q install "pymupdf<1.25" pillow pandas

# OCR opcional (si hay páginas sin texto)
# (descomenta las 2 líneas siguientes si lo necesitas)
# !apt-get -y install tesseract-ocr
# !pip -q install pytesseract


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 37.0 MB/s eta 0:00:00


# Parte 1
Se limpia carpetas antes de guardar los nuevos pdf's.

In [4]:
# Parte 1: limpieza completa para comenzar ingestion nueva
from pathlib import Path
import shutil, os, re

# Carpetas de trabajo
folder_pdf = Path("/content/original_pdfs")
folder_img = Path("/content/pdf_pages_benef_finales")

# Limpia carpetas si existen
for p in [folder_pdf, folder_img]:
    if p.exists():
        shutil.rmtree(p)
    p.mkdir(parents=True, exist_ok=True)

# Limpia PDFs sueltos en /content y en /content/sample_data para evitar duplicados/confusiones
for wild in ["*.pdf", "* (1).pdf", "* (2).pdf"]:
    for f in Path("/content").glob(wild):
        try:
            f.unlink()
        except Exception:
            pass
    sd = Path("/content/sample_data")
    if sd.exists():
        for f in sd.glob(wild):
            try:
                f.unlink()
            except Exception:
                pass

print("Entorno limpio: listo para subir PDFs.")


Entorno limpio: listo para subir PDFs.


# Parte 2
Se suben los pdf directo en Colab y se guardan los pdf en la carpeta ´folder_pdf´ que es original_pdfs

In [5]:
# Parte 2: subir y organizar archivos
from pathlib import Path
from google.colab import files
import shutil, os

# Carpetas
folder_pdf = Path("/content/original_pdfs")            # PDFs fuente
folder_img = Path("/content/pdf_pages_benef_finales")  # imágenes de páginas detectadas

# (Re)crear limpio
for p in [folder_pdf, folder_img]:
    if p.exists():
        shutil.rmtree(p)
    p.mkdir(parents=True, exist_ok=True)

print("Sube tus PDFs (puedes seleccionar varios a la vez)…")
uploaded = files.upload()  # selector de archivos

# Guardar en folder_pdf
for fname, data in uploaded.items():
    if not fname.lower().endswith(".pdf"):
        print(f"⚠️  Omitido (no es PDF): {fname}")
        continue
    out_path = folder_pdf / fname
    with open(out_path, "wb") as f:
        f.write(data)
    print(f"✅ Guardado: {out_path}")

# Verificación básica
print("\nPDFs en", folder_pdf, ":")
print(sorted([p.name for p in folder_pdf.glob("*.pdf")])[:10],
      "..." if len(list(folder_pdf.glob('*.pdf'))) > 10 else "")


Sube tus PDFs (puedes seleccionar varios a la vez)…


Saving 998765477.pdf to 998765477.pdf
Saving 70618760K.pdf to 70618760K.pdf
Saving 77123458K.pdf to 77123458K.pdf
Saving 737543456.pdf to 737543456.pdf
Saving 761234589.pdf to 761234589.pdf
Saving 763456476.pdf to 763456476.pdf
Saving 778765477.pdf to 778765477.pdf
Saving 778765522.pdf to 778765522.pdf
Saving 789654322.pdf to 789654322.pdf
Saving 887655999.pdf to 887655999.pdf
Saving 987654321.pdf to 987654321.pdf
Saving 998765437.pdf to 998765437.pdf
✅ Guardado: /content/original_pdfs/998765477.pdf
✅ Guardado: /content/original_pdfs/70618760K.pdf
✅ Guardado: /content/original_pdfs/77123458K.pdf
✅ Guardado: /content/original_pdfs/737543456.pdf
✅ Guardado: /content/original_pdfs/761234589.pdf
✅ Guardado: /content/original_pdfs/763456476.pdf
✅ Guardado: /content/original_pdfs/778765477.pdf
✅ Guardado: /content/original_pdfs/778765522.pdf
✅ Guardado: /content/original_pdfs/789654322.pdf
✅ Guardado: /content/original_pdfs/887655999.pdf
✅ Guardado: /content/original_pdfs/987654321.pdf
✅ Gua

Ver cuantos pdf se subieron exitosamente :)

In [6]:
# Parte 2.1 — Resumen de subida de PDFs
from pathlib import Path
import re

# Usa la misma carpeta y la misma función de limpieza que en la Parte 2
# folder_pdf = Path("/content/original_pdfs")

def clean_name(fname: str) -> str:
    p = Path(fname)
    stem = re.sub(r"\s\(\d+\)$", "", p.stem)  # quita ' (1)', ' (2)', etc. al final
    return f"{stem}.pdf"

# 1) ¿Cuántos PDFs intenté subir? (desde 'uploaded' de la Parte 2)
intentados_raw = [k for k in uploaded.keys() if k.lower().endswith(".pdf")]
intentados_limpios = [clean_name(k) for k in intentados_raw]

# 2) ¿Cuántos quedaron guardados en la carpeta?
guardados = sorted([p.name for p in folder_pdf.glob("*.pdf")])

# 3) Resumen
set_intentados = set(intentados_limpios)
set_guardados  = set(guardados)

exitosos = sorted(set_intentados & set_guardados)
faltantes = sorted(set_intentados - set_guardados)
extras    = sorted(set_guardados - set_intentados)  # (p.ej. si ya había archivos previos)

print(f"Intentaste subir: {len(intentados_limpios)} PDF(s)")
print(f"Se subieron exitosamente: {len(exitosos)} PDF(s)")

if exitosos:
    print("✓ Guardados:", exitosos)
if faltantes:
    print("⚠️  Faltaron (no se guardaron):", faltantes)
if extras:
    print("ℹ️  Extras en la carpeta (no venían en esta subida):", extras)


Intentaste subir: 12 PDF(s)
Se subieron exitosamente: 12 PDF(s)
✓ Guardados: ['70618760K.pdf', '737543456.pdf', '761234589.pdf', '763456476.pdf', '77123458K.pdf', '778765477.pdf', '778765522.pdf', '789654322.pdf', '887655999.pdf', '987654321.pdf', '998765437.pdf', '998765477.pdf']


parte 2 ejecutada con éxito

# Parte 3: OCR


Instalo librerias de OCR

In [7]:
# OCR con IA (Tesseract + idioma español)
!apt-get -y install tesseract-ocr tesseract-ocr-spa >/dev/null
!pip -q install pytesseract


Busca “Identificación de Beneficiarios Finales” y algunas pistas de columnas.
busca la pagina con OCR con libreria *pytesseract*.

In [8]:
# Parte 3 (OCR): detectar página(s) de "Identificación de Beneficiarios Finales" y exportar PNG
import fitz, re, unicodedata, pytesseract #pytesseract -> ocr
from PIL import Image
from pathlib import Path

# Reusa estas variables creadas en la Parte 2
# folder_pdf = Path("/content/original_pdfs")
# folder_img = Path("/content/pdf_pages_benef_finales")

DPI  = 220    ##mejor nitidez en imagen
zoom = DPI/72
MAT  = fitz.Matrix(zoom, zoom)

def norm(s: str) -> str:
    s = s or ""
    s = s.lower()
    s = unicodedata.normalize("NFD", s)
    return "".join(ch for ch in s if unicodedata.category(ch) != "Mn")

# palabras claves en la seccion "Identificación de Beneficiarios Finales"
SECTION_HINTS = tuple(map(norm, [
    "Identificación de Beneficiarios Finales",
    "Identificacion de Beneficiarios Finales",
    "Identificación de Beneficiario Final",
    "Identificacion de Beneficiario Final"
]))

#las columnas que tiene el cuadro dicen "Rut/pasaporte, Porcentaje de participación", etc
COL_HINTS = tuple(map(norm, [
    "RUT/Pasaporte","RUT","R.U.T","Pasaporte",
    "Nombre Completo","Nacionalidad","Fecha de Nacimiento",
    "Domicilio, Ciudad y País","Participación","Participacion"
]))

def page_text_pdf(doc, i) -> str:
    return doc.load_page(i).get_text("text") or ""
#usamos OCR para detectar
def page_text_ocr(doc, i) -> str:
    # Render y OCR en español
    pix = doc.load_page(i).get_pixmap(matrix=MAT, alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    return pytesseract.image_to_string(img, lang="spa")

def score_text(txt_norm: str) -> int:
    score = 0
    if any(h in txt_norm for h in SECTION_HINTS):
        score += 5
    for c in COL_HINTS:
        if c in txt_norm:
            score += 1
    return score

def pick_pages(doc):
    """
    Estrategia híbrida:
    - Calcula score con texto PDF.
    - Si el score es 0 o bajo (<5), intenta OCR y recalcula.
    - Devuelve páginas con score>=5; si ninguna, la mejor.
    """
    chosen = []
    best = (-1, -1)  # (score, index)

    for i in range(doc.page_count):
        t_pdf = page_text_pdf(doc, i)
        s_pdf = score_text(norm(t_pdf))

        if s_pdf < 5:
            t_ocr = page_text_ocr(doc, i)
            s_ocr = score_text(norm(t_ocr))
            s = max(s_pdf, s_ocr)
        else:
            s = s_pdf

        if s >= 5:
            chosen.append(i)
        if s > best[0]:
            best = (s, i)

    if not chosen and best[1] >= 0:
        chosen = [best[1]]
    return sorted(set(chosen))

#exporta las paginas que detecto y las guarda en las subfolder con el nombre de cada pdf
def export_detected_pages():
    total = 0
    for pdf in sorted(folder_pdf.glob("*.pdf")):
        out_dir = folder_img / pdf.stem
        out_dir.mkdir(parents=True, exist_ok=True)

        with fitz.open(pdf) as doc:
            pages = pick_pages(doc)
            for i in pages:
                # Render a PNG y nombrar como <PDF>_p####.png
                pix = doc.load_page(i).get_pixmap(matrix=MAT, alpha=False)
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                img.save(out_dir / f"{pdf.stem}_p{i+1:04d}.png", "PNG", optimize=True)
                total += 1
        print(f"✔ {pdf.name}: {len(pages)} imagen(es) → {out_dir}")
    print("Total imágenes exportadas:", total)

export_detected_pages()


✔ 70618760K.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/70618760K
✔ 737543456.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/737543456
✔ 761234589.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/761234589
✔ 763456476.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/763456476
✔ 77123458K.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/77123458K
✔ 778765477.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/778765477
✔ 778765522.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/778765522
✔ 789654322.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/789654322
✔ 887655999.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/887655999
✔ 987654321.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/987654321
✔ 998765437.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/998765437
✔ 998765477.pdf: 1 imagen(es) → /content/pdf_pages_benef_finales/998765477
Total imágenes exportadas: 12


hasta aqui vamos excelente, detecto las hojas acorde a lo esperado con IA especificamente OCR libreria **pytesseract**, todo bien :)

## Encuentra la página con la sección.


leemos las imagenes rescatadas en la carpeta pdf_pages_benef_finales y extraemos los datos que queremos trabajar

calibracion manual

In [49]:
# @title
# ================== TUNER MANUAL (limpia carpetas + nuevo overlay) ==================
# - Borra /content/_bf_debug_tune y /content/_bf_debug
# - Dibuja SOLO rectángulos de RUT (rojo) y Participación (azul)
# - Guarda overlay y recortes para inspección visual

import shutil
from pathlib import Path
from PIL import Image, ImageDraw

# -------- 1) LIMPIAR CARPETAS DE DEBUG --------
for d in ["/content/_bf_debug_tune", "/content/_bf_debug"]:
    p = Path(d)
    if p.exists():
        shutil.rmtree(p)
    p.mkdir(parents=True, exist_ok=True)

# -------- 2) ELEGIR IMAGEN A CALIBRAR --------
DOC = "70618760K"                # carpeta dentro de folder_img
IMG = "70618760K_p0005.png"      # nombre de la imagen dentro de esa carpeta

# -------- 3) AJUSTES MANUALES (MUEVE ESTOS NÚMEROS) --------
# Verticales (0 = arriba, 1 = abajo). Para SUBIR: baja ambos.
ROW_TOP_RUT, ROW_BOTTOM_RUT = 0.18, 0.54
ROW_TOP_PCT, ROW_BOTTOM_PCT = 0.18, 0.54

# Horizontales:
# RUT → para mover a la DERECHA, sube ambos; para ANGOSTAR, acerca x0 y x1.
# (ejemplo: de (0.13, 0.27) pasa a (0.16, 0.26) → se corre a la derecha y se angosta)
COL_RUT_DEF  = (0.22, 0.32)

# Participación → para mover a la IZQUIERDA, baja ambos (x0, x1)
COL_PART_DEF = (0.62, 0.74)

# -------- 4) DIBUJAR Y GUARDAR --------
root = folder_img / DOC
img_path = root / IMG
assert img_path.exists(), f"No existe: {img_path}"

DBG_DIR_TUNE = Path("/content/_bf_debug_tune") / DOC
DBG_DIR_TUNE.mkdir(parents=True, exist_ok=True)

def _to_px(W, H, col, row):
    x0r, x1r = col
    y0r, y1r = row
    return int(W*x0r), int(H*y0r), int(W*x1r), int(H*y1r)

im = Image.open(img_path).convert("RGB")
W, H = im.size
ov = im.copy()
draw = ImageDraw.Draw(ov)

# RUT (rojo)
rut_box = _to_px(W, H, COL_RUT_DEF, (ROW_TOP_RUT, ROW_BOTTOM_RUT))
draw.rectangle(rut_box, outline=(255,0,0), width=3)

# Participación (azul)
pct_box = _to_px(W, H, COL_PART_DEF, (ROW_TOP_PCT, ROW_BOTTOM_PCT))
draw.rectangle(pct_box, outline=(0,128,255), width=3)

# Guardar overlay y recortes
overlay_path = DBG_DIR_TUNE / f"{IMG}_overlay.png"
rut_path     = DBG_DIR_TUNE / f"{IMG}_RUT.png"
pct_path     = DBG_DIR_TUNE / f"{IMG}_PCT.png"

ov.save(overlay_path)
im.crop(rut_box).save(rut_path)
im.crop(pct_box).save(pct_path)

print("Guardado:")
print("  Overlay:", overlay_path)
print("  RUT recorte:", rut_path)
print("  PCT recorte:", pct_path)
print("\nTIP:")
print("- Para mover el ROJO a la DERECHA: sube ambos en COL_RUT_DEF (x0, x1).")
print("- Para ANGOSTAR el ROJO: acerca x0 y x1 (p.ej. 0.17, 0.25).")
print("- Para mover el AZUL a la IZQUIERDA: baja ambos en COL_PART_DEF.")
print("- Para SUBIR ambas bandas: baja ROW_TOP_* y ROW_BOTTOM_*.")


Guardado:
  Overlay: /content/_bf_debug_tune/70618760K/70618760K_p0005.png_overlay.png
  RUT recorte: /content/_bf_debug_tune/70618760K/70618760K_p0005.png_RUT.png
  PCT recorte: /content/_bf_debug_tune/70618760K/70618760K_p0005.png_PCT.png

TIP:
- Para mover el ROJO a la DERECHA: sube ambos en COL_RUT_DEF (x0, x1).
- Para ANGOSTAR el ROJO: acerca x0 y x1 (p.ej. 0.17, 0.25).
- Para mover el AZUL a la IZQUIERDA: baja ambos en COL_PART_DEF.
- Para SUBIR ambas bandas: baja ROW_TOP_* y ROW_BOTTOM_*.


probar la calibracion con todos los pdf

In [50]:
# @title
# ================== DIAGNÓSTICO GLOBAL CON BANDAS FIJAS ==================
# Guarda overlays y recortes en /content/_bf_debug/<doc>/
# Usa tus parámetros calibrados manualmente.

import shutil, os
from pathlib import Path
from PIL import Image, ImageDraw
import pandas as pd

# --- carpeta con imágenes del paso 3 ---
try:
    folder_img
except NameError:
    folder_img = Path("/content/pdf_pages_benef_finales")
folder_img = Path(folder_img)

# --- limpiar salida ---
DBG_DIR = Path("/content/_bf_debug")
if DBG_DIR.exists():
    shutil.rmtree(DBG_DIR)
DBG_DIR.mkdir(parents=True, exist_ok=True)

# ====== PARÁMETROS (tus calibraciones) ======
# Verticales (0=arriba, 1=abajo)
ROW_TOP_RUT, ROW_BOTTOM_RUT = 0.18, 0.54
ROW_TOP_PCT, ROW_BOTTOM_PCT = 0.18, 0.54

# Horizontales
COL_RUT_DEF  = (0.22, 0.32)
COL_PART_DEF = (0.62, 0.74)

# ====== helpers ======
def _to_px(W, H, col, row):
    x0r, x1r = col
    y0r, y1r = row
    return int(W*x0r), int(H*y0r), int(W*x1r), int(H*y1r)

rows = []

# ====== recorrer todas las imágenes ======
for sub in sorted(p for p in folder_img.iterdir() if p.is_dir()):
    out_dir = DBG_DIR / sub.name
    out_dir.mkdir(parents=True, exist_ok=True)

    imgs = sorted(list(sub.glob("*.png")) + list(sub.glob("*.jpg")) + list(sub.glob("*.jpeg")))
    for img_path in imgs:
        im = Image.open(img_path).convert("RGB")
        W, H = im.size
        ov = im.copy()
        draw = ImageDraw.Draw(ov)

        rut_box = _to_px(W, H, COL_RUT_DEF,  (ROW_TOP_RUT, ROW_BOTTOM_RUT))
        pct_box = _to_px(W, H, COL_PART_DEF, (ROW_TOP_PCT, ROW_BOTTOM_PCT))

        # dibujar
        draw.rectangle(rut_box, outline=(255,0,0), width=3)       # rojo RUT/P
        draw.rectangle(pct_box, outline=(0,128,255), width=3)     # azul %
        # guardar
        (out_dir / f"{img_path.name}_overlay.png").write_bytes(ov.tobytes() if False else b"")
        # PIL necesita save normal:
        ov.save(out_dir / f"{img_path.name}_overlay.png")
        im.crop(rut_box).save(out_dir / f"{img_path.name}_RUT.png")
        im.crop(pct_box).save(out_dir / f"{img_path.name}_PCT.png")

        rows.append({
            "doc": sub.name,
            "img": img_path.name,
            "W": W, "H": H,
            "ROW_RUT": (ROW_TOP_RUT, ROW_BOTTOM_RUT),
            "COL_RUT": COL_RUT_DEF,
            "ROW_PCT": (ROW_TOP_PCT, ROW_BOTTOM_PCT),
            "COL_PCT": COL_PART_DEF,
            "overlay": str((out_dir / f"{img_path.name}_overlay.png")),
        })

diag = pd.DataFrame(rows)
print(f"Listo. Revisa overlays en: {DBG_DIR}")
display(diag.head(30))


Listo. Revisa overlays en: /content/_bf_debug


,doc,img,W,H,ROW_RUT,COL_RUT,ROW_PCT,COL_PCT,overlay
0,70618760K,70618760K_p0005.png,1820,2573,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/70618760K/70618760K_p0005.p...
1,737543456,737543456_p0015.png,1820,2573,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/737543456/737543456_p0015.p...
2,761234589,761234589_p0008.png,1820,2573,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/761234589/761234589_p0008.p...
3,763456476,763456476_p0002.png,1820,2573,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/763456476/763456476_p0002.p...
4,77123458K,77123458K_p0008.png,1870,2420,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/77123458K/77123458K_p0008.p...
5,778765477,778765477_p0006.png,1820,2573,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/778765477/778765477_p0006.p...
6,778765522,778765522_p0012.png,1820,2573,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/778765522/778765522_p0012.p...
7,789654322,789654322_p0001.png,1820,2573,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/789654322/789654322_p0001.p...
8,887655999,887655999_p0005.png,1820,2573,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/887655999/887655999_p0005.p...
9,987654321,987654321_p0008.png,1820,2573,"(0.18, 0.54)","(0.22, 0.32)","(0.18, 0.54)","(0.62, 0.74)",/content/_bf_debug/987654321/987654321_p0008.p...


intendo mil de deteccion de datos

In [9]:
# ================== EXTRACCIÓN MIN (LEE TODOS LOS PDFs DESDE _bf_debug) ==================
# Salida: df_min (Rut_PJ, Rut_bf, porc_part, Estado)

import re, pandas as pd
from pathlib import Path

# --- Rutas ---
DEBUG_DIR = Path("/content/_bf_debug")

# --- Patrones principales ---
RUT_FLEX = r"\d{1,2}\.?\d{3}\.?\d{3}-?[0-9Kk]"          # rut flexible
ID_ANY   = rf"(?:{RUT_FLEX}|[0-9A-Za-z]{{7,}})"         # pasaporte >= 7
CI_TAG   = r"C[Il1]"                                    # CI tolerante (CI/Cl/C1)

# --- Normalizadores / parsers ---
def normalize_rut_pj_11(s: str) -> str:
    s = re.sub(r"[^0-9Kk]", "", s or "")
    if not s: return ""
    return s.replace("k","K").rjust(11, "0")[:11]

def normalize_rut_bf_text(txt: str) -> str:
    """Deja 'CI 12345678-9' o 'P ABC12345...' si es posible."""
    if not txt: return ""
    t = " ".join(str(txt).split())
    # 1) patrón CI|P + ID
    m = re.search(rf"\b({CI_TAG}|P)\b\W+({ID_ANY})", t, flags=re.I)
    if m:
        tipo = m.group(1).upper()
        tipo = "CI" if tipo.startswith(("CI","C","CL","C1")) else "P"
        ide  = m.group(2)
        return f"{tipo} {ide}"
    # 2) rescate por RUT o pasaporte largo
    m2 = re.search(ID_ANY, t)
    if m2:
        ide = m2.group(0)
        if re.fullmatch(RUT_FLEX, ide, flags=re.I):
            return f"CI {ide}"
        return f"P {ide}"
    return ""

# --- Parser ESTRICTO de porcentaje (usa el parche que corrige %→8/º/o/O) ---
def _normalize_pct_text(line: str) -> str:
    txt = str(line).replace(" ", "")
    txt = re.sub(r"(?<=\d)[8ºoO°]$", "%", txt)
    txt = re.sub(r"(?<=\d)[8ºoO°](?=\D|$)", "%", txt)
    txt = re.sub(r"(\d{1,3}(?:[.,]\d+)?)[8ºoO°]$", r"\1%", txt)
    return txt

def parse_percent_strict(raw):
    s = _normalize_pct_text(str(raw))
    if "%" not in s:
        return None
    s = s.replace("%", "").replace(",", ".")
    m = re.search(r"\d{1,3}(?:\.\d+)?$", s)
    if not m:
        return None
    try:
        v = float(m.group(0))
    except:
        return None
    if v <= 0 or v > 100:
        return None
    return v

# --- Extractores desde recortes (tienen que existir de tu paso 4A global) ---
from PIL import Image, ImageOps
import pytesseract

def _ocr_data(img, psm="11", whitelist=None):
    cfg = f"--psm {psm}"
    if whitelist:
        cfg += f" -c tessedit_char_whitelist={whitelist}"
    df = pytesseract.image_to_data(
        img, lang="spa", config=cfg, output_type=pytesseract.Output.DATAFRAME
    )
    df = df.dropna(subset=["text"])
    df["text"] = df["text"].astype(str).str.strip()
    return df[df["text"]!=""]

def _group_by_y(df, tol=14):
    rows = []
    if df.empty: return rows
    tmp = df.sort_values("top").copy()
    g = [tmp.iloc[0]]; g_top = float(tmp.iloc[0]["top"])
    for i in range(1, len(tmp)):
        r = tmp.iloc[i]
        if abs(float(r["top"]) - g_top) <= tol:
            g.append(r)
        else:
            rows.append(pd.DataFrame(g))
            g = [r]; g_top = float(r["top"])
    rows.append(pd.DataFrame(g))
    return rows

def extract_rut_lines(rut_png: Path):
    pil = Image.open(rut_png).convert("RGB")
    pil = ImageOps.autocontrast(pil)
    df  = _ocr_data(pil, psm="11", whitelist="0123456789Kk- CIPcip")
    out = []
    for g in _group_by_y(df, tol=14):
        toks = g.sort_values("left")["text"].tolist()
        line = " ".join(toks)
        if re.search(r"rut\s*/?\s*pasap", line, flags=re.I):
            continue
        out.append(normalize_rut_bf_text(line))
    # filtra vacíos
    return [x for x in out if x]

def extract_pct_lines(pct_png: Path):
    pil = Image.open(pct_png).convert("RGB")
    pil = ImageOps.autocontrast(pil)
    # dos pasadas
    lines = []
    for p in ("11","6"):
        df = _ocr_data(pil, psm=p, whitelist="0123456789.,%oO8º°")
        for g in _group_by_y(df, tol=12):
            toks = g.sort_values("left")["text"].tolist()
            line = " ".join(toks)
            if re.search(r"participa", line, flags=re.I):
                continue
            lines.append(line)
    # normaliza y devuelve primer % válido por fila
    out = []
    for ln in lines:
        txt = _normalize_pct_text(ln)
        # toma el primer % “bien formado”
        best = None
        for m in re.finditer(r"\d{1,3}(?:[.,]\d+)?%?", txt):
            seg = m.group(0)
            if not seg.endswith("%"):
                continue
            if parse_percent_strict(seg) is not None:
                best = seg
                break
        if best:
            out.append(best)
    return out

# ---------------- LOOP ROBUSTO (lee TODO _bf_debug) ----------------
records = []
docs = sorted([p for p in DEBUG_DIR.iterdir() if p.is_dir()])
if not docs:
    print("⚠️ No hay recortes en /content/_bf_debug. Genera primero con el diagnóstico global.")
else:
    for d in docs:
        rut_pj_raw = d.name
        rut_pj     = normalize_rut_pj_11(rut_pj_raw)

        # toma todos los pares *_RUT.png ↔ *_PCT.png
        rut_imgs = sorted(d.glob("*_RUT.png"))
        if not rut_imgs:
            # no hay ninguna imagen de RUT para este doc
            records.append({"Rut_PJ": rut_pj, "Rut_bf": "", "porc_part": "", "Estado": "Incidencia: sin _RUT.png"})
            continue

        ok_rows = 0
        for rut_png in rut_imgs:
            base = rut_png.name[:-8]  # quita "_RUT.png"
            pct_png = d / f"{base}_PCT.png"
            if not pct_png.exists():
                # no hay pareja de porcentaje para esta página
                records.append({"Rut_PJ": rut_pj, "Rut_bf": "", "porc_part": "", "Estado": f"Incidencia: falta {pct_png.name}"})
                continue

            ids  = extract_rut_lines(rut_png)
            pcts = extract_pct_lines(pct_png)

            # Emparejar “a la fuerza” por posición (arriba->abajo):
            n = max(len(ids), len(pcts))
            if n == 0:
                records.append({"Rut_PJ": rut_pj, "Rut_bf": "", "porc_part": "", "Estado": "Incidencia: sin filas legibles en esta página"})
                continue

            for i in range(n):
                rid  = ids[i]  if i < len(ids)  else ""
                praw = pcts[i] if i < len(pcts) else ""
                ok_id  = bool(normalize_rut_bf_text(rid))
                ok_pct = parse_percent_strict(praw) is not None
                estado = "Success" if (ok_id and ok_pct) else "Incidencia"
                if estado == "Success":
                    ok_rows += 1
                records.append({"Rut_PJ": rut_pj, "Rut_bf": rid, "porc_part": praw, "Estado": estado})

        if ok_rows == 0:
            # deja un rastro para este doc si ninguna página dio fila válida
            records.append({"Rut_PJ": rut_pj, "Rut_bf": "", "porc_part": "", "Estado": "Incidencia: documento sin filas válidas"})

df_min = pd.DataFrame.from_records(records, columns=["Rut_PJ","Rut_bf","porc_part","Estado"])
print(f"Docs encontrados: {len(docs)}  |  Filas extraídas: {len(df_min)}")
display(df_min.head(50))


FileNotFoundError: [Errno 2] No such file or directory: '/content/_bf_debug'

In [53]:
# @title
# ========= PARCHE COLUMNA PORCENTAJE (mejor OCR + normalización de "%") =========
import numpy as np, cv2, re
from PIL import Image, ImageOps
import pytesseract

try:
    import easyocr
    _easy = easyocr.Reader(['es','en'], gpu=False)
except Exception:
    _easy = None

def _pil2cv(pil):
    return cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)

def _prep_pct(pil):
    """Upscale + denoise + Otsu para que el % se separe de los dígitos."""
    im = ImageOps.autocontrast(pil)
    cv = _pil2cv(im)
    # Escalar 1.8x (mejora glyphs pequeños como '%')
    cv = cv2.resize(cv, None, fx=1.8, fy=1.8, interpolation=cv2.INTER_CUBIC)
    g  = cv2.cvtColor(cv, cv2.COLOR_BGR2GRAY)
    g  = cv2.bilateralFilter(g, 9, 75, 75)
    _, th = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # leve morfología para cerrar puntos del %
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, k, iterations=1)
    return Image.fromarray(th)

def _ocr_data(img, psm="11", whitelist=None):
    cfg = f"--psm {psm}"
    if whitelist:
        cfg += f" -c tessedit_char_whitelist={whitelist}"
    df = pytesseract.image_to_data(
        img, lang="spa", config=cfg, output_type=pytesseract.Output.DATAFRAME
    )
    df = df.dropna(subset=["text"])
    df["text"] = df["text"].astype(str).str.strip()
    df = df[df["text"]!=""]
    if "conf" in df.columns:
        df["conf"] = pd.to_numeric(df["conf"], errors="coerce").fillna(-1)
        df = df[df["conf"] > 20]
    return df

def _group_by_y(df, tol=12):
    rows = []
    if df.empty: return rows
    tmp = df.sort_values("top").copy()
    g = [tmp.iloc[0]]; g_top = float(tmp.iloc[0]["top"])
    for i in range(1, len(tmp)):
        r = tmp.iloc[i]
        if abs(float(r["top"]) - g_top) <= tol:
            g.append(r)
        else:
            rows.append(pd.DataFrame(g))
            g = [r]; g_top = float(r["top"])
    rows.append(pd.DataFrame(g))
    return rows

def _normalize_pct_text(line: str) -> str:
    """
    Corrige confusiones comunes: % → {8,º,o,O,°}. Requiere % tras normalizar.
    Casos: '15,91 8' -> '15,91%', '10º'->'10%', '17,1o'->'17,1%'.
    """
    txt = line.replace(" ", "")
    # si hay '8' justo al final de un número, cámbialo a %
    txt = re.sub(r"(?<=\d)[8ºoO°]$", "%", txt)
    # si hay número seguido de uno de esos símbolos, conviértelo
    txt = re.sub(r"(?<=\d)[8ºoO°](?=\D|$)", "%", txt)
    # en algunos casos tesseract separa: '15,91' '8' -> únelos
    txt = re.sub(r"(\d{1,3}(?:[.,]\d+)?)[8ºoO°]$", r"\1%", txt)
    return txt

def parse_percent_strict(raw):
    """
    Solo acepta si hay '%' (después de normalizar) y 0 < v <= 100.
    Evita ruidos tipo 210.
    """
    s = _normalize_pct_text(str(raw))
    if "%" not in s:
        return None
    s = s.replace("%", "").replace(",", ".")
    m = re.search(r"\d{1,3}(?:\.\d+)?$", s)
    if not m:
        return None
    try:
        v = float(m.group(0))
    except:
        return None
    if v <= 0 or v > 100:
        return None
    return v

def extract_pct_lines(pct_png: Path):
    """Devuelve lista de porcentajes crudos (con %) por fila ya normalizados."""
    pil0 = Image.open(pct_png).convert("RGB")
    pil  = _prep_pct(pil0)

    # Tesseract (dos pasadas)
    parts = []
    for p in ("11","6"):
        df = _ocr_data(pil, psm=p, whitelist="0123456789.,%oO8º°")
        for g in _group_by_y(df, tol=12):
            toks = g.sort_values("left")["text"].tolist()
            line = " ".join(toks)
            if re.search(r"participa", line, flags=re.I):
                continue
            parts.append(line)

    # EasyOCR respaldo si hay pocas
    if _easy is not None and len(parts) < 2:
        arr = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)
        txts = _easy.readtext(arr, detail=0, paragraph=True)
        parts.extend(txts)

    # Normalizar y seleccionar primer % válido por fila
    out = []
    for line in parts:
        norm = _normalize_pct_text(line)
        miter = re.finditer(r"\d{1,3}(?:[.,]\d+)?%?", norm)
        val_txt = None
        for m in miter:
            seg = m.group(0)
            if not seg.endswith("%"):
                continue  # exigimos % tras normalizar
            if parse_percent_strict(seg) is not None:
                val_txt = seg
                break
        if val_txt:
            out.append(val_txt)
    return out


In [54]:
# @title
# <-- deja tu loop tal cual usa parse_percent(...)
def parse_percent(x):
    return parse_percent_strict(x)


In [55]:
# @title
# ================== EXTRACCIÓN ROBUSTA DESDE _bf_debug (RUT y % SOLAMENTE) ==================
import re, cv2, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image, ImageOps
import pytesseract

DEBUG_DIR = Path("/content/_bf_debug")

# -------------------- Configurables --------------------
Y_TOL = 12          # tolerancia vertical para agrupar tokens en una misma fila
MIN_PCT = 0.1       # descarta "0%" o ruidos muy pequeños
MAX_PCT = 100.0     # valor máximo razonable
PSMS = ["11","6"]   # dos pasadas

# -------------------- Patrones --------------------
RUT_FLEX = r"\d{1,2}\.?\d{3}\.?\d{3}-?[0-9Kk]"
ID_ANY   = rf"(?:{RUT_FLEX}|[0-9A-Za-z]{{7,}})"      # pasaporte >=7
CI_TAG   = r"C[Il1]"                                 # CI tolerante
PCT_ANY  = r"\d{1,3}(?:[.,]\d+)?%"

# -------------------- Utilidades --------------------
def _pil2cv(pil):
    return cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)

def _prep_rut(pil):
    # contraste + gris + bin adaptativa suave para números
    im = ImageOps.autocontrast(pil)
    cv = _pil2cv(im)
    g  = cv2.cvtColor(cv, cv2.COLOR_BGR2GRAY)
    g  = cv2.adaptiveThreshold(g, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 31, 8)
    return Image.fromarray(g)

def _prep_pct(pil):
    # contraste + bilateral (suaviza ruido) + Otsu para dígitos %
    im = ImageOps.autocontrast(pil)
    cv = _pil2cv(im)
    g  = cv2.cvtColor(cv, cv2.COLOR_BGR2GRAY)
    g  = cv2.bilateralFilter(g, 7, 50, 50)
    _, th = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return Image.fromarray(th)

def _ocr_data(img, psm="11", whitelist=None):
    cfg = f"--psm {psm}"
    if whitelist:
        cfg += f" -c tessedit_char_whitelist={whitelist}"
    df = pytesseract.image_to_data(
        img, lang="spa", config=cfg, output_type=pytesseract.Output.DATAFRAME
    )
    df = df.dropna(subset=["text"])
    df["text"] = df["text"].astype(str).str.strip()
    df = df[df["text"]!=""]
    # guard rails: quitar conf muy baja
    if "conf" in df.columns:
        df["conf"] = pd.to_numeric(df["conf"], errors="coerce").fillna(-1)
        df = df[df["conf"] > 20]
    return df

def _group_by_y(df, tol=Y_TOL):
    rows = []
    if df.empty: return rows
    tmp = df.sort_values("top").copy()
    g = [tmp.iloc[0]]; g_top = float(tmp.iloc[0]["top"])
    for i in range(1, len(tmp)):
        r = tmp.iloc[i]
        if abs(float(r["top"]) - g_top) <= tol:
            g.append(r)
        else:
            rows.append(pd.DataFrame(g))
            g = [r]; g_top = float(r["top"])
    rows.append(pd.DataFrame(g))
    return rows

def parse_percent(raw):
    raw = str(raw).replace(" ", "")
    raw = raw.replace(",", ".")
    m = re.search(r"\d{1,3}(?:\.\d+)?", raw)
    if not m:
        return None
    try:
        v = float(m.group(0))
    except:
        return None
    if v <= MIN_PCT or v > MAX_PCT:
        return None
    return v

# -------------------- Extractores por columna --------------------
def extract_rut_lines(rut_png: Path):
    pil0 = Image.open(rut_png).convert("RGB")
    pil  = _prep_rut(pil0)

    # Dos pasadas: whitelist para RUT/CI/P
    dfs = []
    for p in PSMS:
        df = _ocr_data(pil, psm=p, whitelist="0123456789Kk- CIPcip")
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True).sort_values(["top","left"])

    out = []
    for g in _group_by_y(df):
        toks = g.sort_values("left")["text"].tolist()
        line = " ".join(toks)

        # saltar header
        if re.search(r"rut\s*/?\s*pasap", line, flags=re.I):
            continue

        # 1) CI|P + ID
        m = re.search(rf"\b({CI_TAG}|P)\b\W+({ID_ANY})", line, flags=re.I)
        if m:
            tipo = m.group(1).upper()
            tipo = "CI" if tipo.startswith(("CI","C","CL","C1")) else "P"
            ide  = m.group(2)
            out.append(f"{tipo} {ide}")
            continue

        # 2) rescate: primer RUT o pasaporte largo en la línea
        m2 = re.search(ID_ANY, line)
        if m2:
            ide = m2.group(0)
            # si no se ve el "CI/P", asumimos CI para RUT y P si es alfanumérico
            if re.fullmatch(RUT_FLEX, ide, flags=re.I):
                out.append(f"CI {ide}")
            else:
                out.append(f"P {ide}")

    return out

def extract_pct_lines(pct_png: Path):
    pil0 = Image.open(pct_png).convert("RGB")
    pil  = _prep_pct(pil0)

    dfs = []
    for p in PSMS:
        df = _ocr_data(pil, psm=p, whitelist="0123456789.,%")
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True).sort_values(["top","left"])

    out = []
    for g in _group_by_y(df):
        toks = g.sort_values("left")["text"].tolist()
        line = " ".join(toks)
        # saltar header
        if re.search(r"participa", line, flags=re.I):
            continue
        # tomar el primer % válido
        for m in re.finditer(PCT_ANY, line):
            v = parse_percent(m.group(0))
            if v is not None:
                # devuelve texto crudo como lo viste (con , o . y con % si estaba)
                out.append(m.group(0))
                break
    return out

# -------------------- Recorrer todo _bf_debug --------------------
records = []
for doc_dir in sorted([p for p in DEBUG_DIR.iterdir() if p.is_dir()]):
    rut_pj = doc_dir.name

    for rut_png in sorted(doc_dir.glob("*_RUT.png")):
        base = rut_png.name[:-8]
        pct_png = doc_dir / f"{base}_PCT.png"
        if not pct_png.exists():
            continue

        ids  = extract_rut_lines(rut_png)
        pcts = extract_pct_lines(pct_png)

        n = max(len(ids), len(pcts))
        if n == 0:
            records.append({"Rut_PJ": rut_pj, "Rut_bf": "", "porc_part": "", "Estado": "Incidencia: sin filas legibles"})
            continue

        for i in range(n):
            rid  = ids[i]  if i < len(ids)  else ""
            praw = pcts[i] if i < len(pcts) else ""
            ok_id  = bool(rid)
            ok_pct = parse_percent(praw) is not None
            estado = "Success" if (ok_id and ok_pct) else "Incidencia"
            records.append({"Rut_PJ": rut_pj, "Rut_bf": rid, "porc_part": praw, "Estado": estado})

df_min = pd.DataFrame.from_records(records, columns=["Rut_PJ","Rut_bf","porc_part","Estado"])
print(f"Filas extraídas: {len(df_min)}")
display(df_min.head(40))


Filas extraídas: 20


,Rut_PJ,Rut_bf,porc_part,Estado
0,70618760K,,,Incidencia: sin filas legibles
1,737543456,,,Incidencia: sin filas legibles
2,761234589,CI 15123456k,10%,Success
3,761234589,,19%,Incidencia
4,763456476,CI 16453627-8,10%,Success
5,77123458K,,,Incidencia: sin filas legibles
6,778765477,,,Incidencia: sin filas legibles
7,778765522,CI 12234581,10%,Success
8,789654322,P 7987651,10%,Success
9,789654322,CI 19876859,"15,91%",Success


no funciono

In [51]:
# @title
# ================== EXTRACCIÓN DESDE RECORTES _bf_debug (RUT y % solamente) ==================
import re, pandas as pd
from pathlib import Path
from PIL import Image, ImageOps
import pytesseract

DEBUG_DIR = Path("/content/_bf_debug")   # aquí guardaste los recortes

# ---------- helpers ----------
RUT_FLEX = r"\d{1,2}\.?\d{3}\.?\d{3}-?[0-9Kk]"  # rut con/sin puntos/guion y DV K
ID_ANY   = rf"(?:{RUT_FLEX}|[0-9A-Za-z]{{7,}})" # pasaporte >=7 como respaldo
CI_TAG   = r"C[Il1]"                             # CI tolerante a OCR
PCT_ANY  = r"\d{1,3}(?:[.,]\d+)?%"

def _ocr_data(img, psm="11"):
    df = pytesseract.image_to_data(
        img, lang="spa", config=f"--psm {psm}",
        output_type=pytesseract.Output.DATAFRAME
    )
    df = df.dropna(subset=["text"])
    df["text"] = df["text"].astype(str).str.strip()
    return df[df["text"]!=""]

def _group_by_y(df, tol=14):
    rows = []
    if df.empty: return rows
    tmp = df.sort_values("top").copy()
    g = [tmp.iloc[0]]; g_top = tmp.iloc[0]["top"]
    for i in range(1, len(tmp)):
        r = tmp.iloc[i]
        if abs(float(r["top"]) - float(g_top)) <= tol:
            g.append(r)
        else:
            rows.append(pd.DataFrame(g))
            g = [r]; g_top = r["top"]
    rows.append(pd.DataFrame(g))
    return rows

def parse_percent(raw):
    raw = str(raw).strip()
    val = raw.replace(" ", "").replace(",", ".").replace("%","")
    val = re.sub(r"[^0-9.]", "", val)
    if val == "":
        return None
    try:
        f = float(val)
    except:
        return None
    return f if 0 < f <= 100 else None

# ---------- extractores columna a columna ----------
def extract_ids_from_rut_col(png_path):
    """Devuelve lista [ 'CI 12345678-9', 'CI 9999999K', 'P ABC123456', ... ] (texto crudo por fila)."""
    pil = Image.open(png_path).convert("RGB")
    pil = ImageOps.autocontrast(pil)
    df  = _ocr_data(pil, psm="11")
    out = []
    for g in _group_by_y(df, tol=14):
        toks = g.sort_values("left")["text"].tolist()
        line = " ".join(toks)
        # quitar header
        if re.search(r"rut\s*/?\s*pasap", line, flags=re.I):
            continue
        # patrón principal: [CI|P] <id>
        m = re.search(rf"\b({CI_TAG}|P)\b\W+({ID_ANY})", line, flags=re.I)
        if m:
            tipo = m.group(1).upper()
            tipo = "CI" if tipo.startswith(("CI","C","CL","C1")) else "P"
            ide  = m.group(2).strip()
            out.append(f"{tipo} {ide}")
            continue
        # fallback: tomar token siguiente si hay CI/P suelto
        for i, t in enumerate(toks):
            if re.fullmatch(rf"({CI_TAG}|P)", t, flags=re.I) and i+1 < len(toks):
                tipo = t.upper()
                tipo = "CI" if tipo.startswith(("CI","C","CL","C1")) else "P"
                ide  = toks[i+1]
                out.append(f"{tipo} {ide}")
                break
    return out

def extract_pcts_from_pct_col(png_path):
    """Devuelve lista de porcentajes crudos (con o sin %) por fila, en orden vertical."""
    pil = Image.open(png_path).convert("RGB")
    pil = ImageOps.autocontrast(pil)
    df  = _ocr_data(pil, psm="11")
    out = []
    for g in _group_by_y(df, tol=14):
        toks = g.sort_values("left")["text"].tolist()
        line = " ".join(toks)
        if re.search(r"participa", line, flags=re.I):  # saltar header
            continue
        m = re.search(PCT_ANY, line)
        if m:
            out.append(m.group(0))
    return out

# ---------- recorrer todos los documentos/recortes ----------
records = []
for doc_dir in sorted([p for p in DEBUG_DIR.iterdir() if p.is_dir()]):
    rut_pj = doc_dir.name  # este ya es el RUT del PDF
    # buscar todos los _RUT.png y emparejar por prefijo
    for rut_png in sorted(doc_dir.glob("*_RUT.png")):
        base = rut_png.name[:-8]  # quita "_RUT.png"
        pct_png = doc_dir / f"{base}_PCT.png"
        if not pct_png.exists():
            continue

        ids  = extract_ids_from_rut_col(rut_png)
        pcts = extract_pcts_from_pct_col(pct_png)

        n = max(len(ids), len(pcts))
        if n == 0:
            records.append({"Rut_PJ": rut_pj, "Rut_bf": "", "porc_part": "",
                            "Estado": "Incidencia: sin filas legibles"})
            continue

        for i in range(n):
            rid  = ids[i]  if i < len(ids)  else ""
            praw = pcts[i] if i < len(pcts) else ""
            estado = "Success" if (rid and praw and parse_percent(praw) is not None) else "Incidencia"
            records.append({
                "Rut_PJ": rut_pj,
                "Rut_bf": rid,
                "porc_part": praw,
                "Estado": estado
            })

df_min = pd.DataFrame.from_records(records, columns=["Rut_PJ","Rut_bf","porc_part","Estado"])
print(f"Filas extraídas: {len(df_min)}")
display(df_min.head(30))


Filas extraídas: 21


,Rut_PJ,Rut_bf,porc_part,Estado
0,70618760K,CI MARI,210%,Incidencia
1,737543456,,210%,Incidencia
2,761234589,,210%,Incidencia
3,763456476,,210%,Incidencia
4,763456476,,510%,Incidencia
5,77123458K,,210%,Incidencia
6,77123458K,,0%,Incidencia
7,778765477,,210%,Incidencia
8,778765522,,210%,Incidencia
9,789654322,CI 18766789K,210%,Incidencia


In [31]:
# @title
# ====================== PASO 4A — DIAGNÓSTICO DE COLUMNAS (SOLO VISUAL) ======================
# Requiere que 'folder_img' ya exista (producido en el Paso 3).
# Salidas:
#  - Carpeta con imágenes: /content/_bf_debug/<doc>/
#  - DataFrame 'diag' con info de columnas usadas y paths de los recortes.

import re, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image, ImageOps, ImageDraw
import pytesseract

# ----------------- PARÁMETROS QUE PUEDES AJUSTAR -----------------
ROW_TOP, ROW_BOTTOM = 0.54, 0.51        # banda vertical del cuerpo de la tabla
MAX_IMGS_PER_DOC = None                 # None = todas las páginas; o pon, p.ej., 2
DBG_DIR = Path("/content/_bf_debug")    # carpeta de salida con recortes/overlays
DBG_DIR.mkdir(parents=True, exist_ok=True)

# Defaults si no hay headers
COL_RUT_DEF  = (0.14, 0.28)             # 2ª columna aproximada
COL_PART_DEF = (0.7, 0.7)             # antepenúltima

# Headers "flexibles" (coinciden aunque estén recortados por el OCR)
HDR_PAT_RUT  = re.compile(r"rut\s*/?\s*pas[aá]p", re.I)   # "rut / pasap..."
HDR_PAT_PART = re.compile(r"participa", re.I)             # "participa..."
SECTION_HINTS = ("identificaci", "benef", "beneficiario")

# ----------------- HELPERS OCR / RECORTES -----------------
def _ocr_data_full(pil, psm="11"):
    df = pytesseract.image_to_data(
        pil, lang="spa", config=f"--psm {psm}",
        output_type=pytesseract.Output.DATAFRAME
    )
    df = df.dropna(subset=["text"])
    df["text"] = df["text"].astype(str).str.strip()
    # bajamos el umbral para ver más candidatos
    df["conf"] = pd.to_numeric(df.get("conf", -1), errors="coerce").fillna(-1)
    return df[(df["text"]!="") & (df["conf"] >= 10)]

def _crop_rel(pil, x0r, y0r, x1r, y1r):
    W, H = pil.size
    box = (int(W*x0r), int(H*y0r), int(W*x1r), int(H*y1r))
    return pil.crop(box), box

def autocalibrate_columns(pil: Image.Image):
    """Devuelve (COL_RUT, COL_PART) en coords relativas [0..1]."""
    W, H = pil.size
    df = _ocr_data_full(ImageOps.autocontrast(pil), psm="11")

    rut_xs, part_xs = [], []
    for _, r in df.iterrows():
        t = r["text"]
        xc = float(r["left"]) + float(r["width"])/2.0
        if HDR_PAT_RUT.search(t):  rut_xs.append(xc)
        if HDR_PAT_PART.search(t): part_xs.append(xc)

    COL_RUT, COL_PART = COL_RUT_DEF, COL_PART_DEF
    col_w = 0.12 * W

    def make_band(xc):
        x0 = max(0, int(xc - col_w*0.55))
        x1 = min(W, int(xc + col_w*0.45))
        return (x0/W, ROW_TOP, x1/W, ROW_BOTTOM)

    if rut_xs:
        xc = float(np.median(rut_xs))
        x0r, _, x1r, _ = make_band(xc)
        COL_RUT = (x0r, x1r)
    else:
        # si no hay header, ensancha un poco la banda por defecto
        COL_RUT = (max(0.10, COL_RUT_DEF[0]), min(0.32, COL_RUT_DEF[1] + 0.04))

    if part_xs:
        xc = float(np.median(part_xs))
        x0r, _, x1r, _ = make_band(xc)
        COL_PART = (x0r, x1r)

    return COL_RUT, COL_PART, len(rut_xs), len(part_xs)

# ----------------- DIAGNÓSTICO: GUARDAR RECORTES/OVERLAYS -----------------
def debug_columns_for_images(img_root: Path, save_dir: Path) -> pd.DataFrame:
    rows = []

    for sub in sorted([p for p in img_root.iterdir() if p.is_dir()]):
        imgs = sorted(list(sub.glob("*.png")) + list(sub.glob("*.jpg")) + list(sub.glob("*.jpeg")))
        if MAX_IMGS_PER_DOC is not None:
            imgs = imgs[:MAX_IMGS_PER_DOC]
        if not imgs:
            continue

        out_dir = save_dir / sub.name
        out_dir.mkdir(parents=True, exist_ok=True)

        for img in imgs:
            pil = Image.open(img).convert("RGB")
            W, H = pil.size

            COL_RUT, COL_PART, rut_hits, part_hits = autocalibrate_columns(pil)

            # recortes
            rut_crop, (rx0, ry0, rx1, ry1) = _crop_rel(pil, COL_RUT[0], ROW_TOP, COL_RUT[1], ROW_BOTTOM)
            pct_crop, (px0, py0, px1, py1) = _crop_rel(pil, COL_PART[0], ROW_TOP, COL_PART[1], ROW_BOTTOM)

            # guardar recortes
            rut_path = out_dir / f"{img.name}_RUT.png"
            pct_path = out_dir / f"{img.name}_PCT.png"
            rut_crop.save(rut_path)
            pct_crop.save(pct_path)

            # overlay
            ov = pil.copy()
            draw = ImageDraw.Draw(ov)
            draw.rectangle([rx0, ry0, rx1, ry1], outline=(255,0,0), width=3)     # rojo = RUT/P
            draw.rectangle([px0, py0, px1, py1], outline=(0,128,255), width=3)   # azul = Participación
            overlay_path = out_dir / f"{img.name}_overlay.png"
            ov.save(overlay_path)

            rows.append({
                "doc": sub.name,
                "img": img.name,
                "W": W, "H": H,
                "rut_hdr_hits": rut_hits,
                "part_hdr_hits": part_hits,
                "COL_RUT": COL_RUT,
                "COL_PART": COL_PART,
                "rut_path": str(rut_path),
                "pct_path": str(pct_path),
                "overlay_path": str(overlay_path),
            })

    diag = pd.DataFrame(rows)
    return diag

# ----------------- EJECUTAR 4A -----------------
DBG_DIR.mkdir(exist_ok=True, parents=True)
diag = debug_columns_for_images(folder_img, DBG_DIR)
print(f"Overlays y recortes guardados en: {DBG_DIR}")
display(diag.head(20))


ValueError: Coordinate 'lower' is less than 'upper'

buscar por ubicación

deteccion de encabezado más sensible

In [16]:
# Reemplaza _ocr_data_full por esta versión:
def _ocr_data_full(pil, psm="11"):
    df = pytesseract.image_to_data(
        pil, lang="spa", config=f"--psm {psm}",
        output_type=pytesseract.Output.DATAFRAME
    )
    df = df.dropna(subset=["text"])
    df["text"] = df["text"].astype(str).str.strip()
    # Umbral más bajo: aceptamos conf >= 10
    df["conf"] = pd.to_numeric(df["conf"], errors="coerce").fillna(-1)
    df = df[(df["text"]!="") & (df["conf"] >= 10)]
    return df


ver si se encontraron los enunciados

In [17]:
# =================== PASO 3.5 — DIAGNÓSTICO DE ENCABEZADOS ===================
# Muestra por imagen:
# - si encontró el header "RUT / Pasaporte" y/o "Participación"
# - qué columnas relativas (COL_RUT, COL_PART) se usarían
# - opcional: guarda overlays con los recortes para inspección visual

from pathlib import Path
from PIL import Image, ImageDraw, ImageFont, ImageOps
import numpy as np
import pandas as pd
import pytesseract, re, os

# Config
MAX_IMGS_PER_DOC = 3       # cuántas imágenes por carpeta (doc) diagnosticar (None = todas)
SAVE_OVERLAYS    = True    # guardar PNGs con rectángulos de los recortes
OVERLAY_DIR      = Path("/content/_debug_headers")
OVERLAY_DIR.mkdir(parents=True, exist_ok=True)

# Usamos los mismos patrones de autocalibrate_columns:
HDR_PAT_RUT  = re.compile(r"rut\s*/?\s*pasaporte", re.I)
HDR_PAT_PART = re.compile(r"participaci[oó]n", re.I)

def _ocr_data_full_diag(pil, psm="11"):
    df = pytesseract.image_to_data(
        pil, lang="spa", config=f"--psm {psm}",
        output_type=pytesseract.Output.DATAFRAME
    )
    df = df.dropna(subset=["text"])
    df["text"] = df["text"].astype(str).str.strip()
    df["conf"] = pd.to_numeric(df.get("conf", -1), errors="coerce").fillna(-1)
    df = df[(df["text"]!="") & (df["conf"] >= 10)]
    return df

def probe_headers(img_path: Path):
    pil = Image.open(img_path).convert("RGB")
    W, H = pil.size
    df = _ocr_data_full_diag(ImageOps.autocontrast(pil), psm="11")

    rut_xs, part_xs = [], []
    for _, r in df.iterrows():
        t = r["text"]
        xc = r["left"] + r["width"]/2
        if HDR_PAT_RUT.search(t):
            rut_xs.append(xc)
        if HDR_PAT_PART.search(t):
            part_xs.append(xc)

    # Llama a tu función real de autocalibración
    COL_RUT, COL_PART = autocalibrate_columns(pil)  # devuelve (x0r,x1r), (x0r,x1r)

    # Overlays opcionales
    if SAVE_OVERLAYS:
        im2 = pil.copy()
        draw = ImageDraw.Draw(im2)
        # Convertir bandas relativas a pixeles
        def to_px(col):
            x0r, x1r = col
            x0, x1 = int(W*x0r), int(W*x1r)
            y0, y1 = int(H*ROW_TOP), int(H*ROW_BOTTOM)
            return x0, y0, x1, y1

        # Dibujar recortes propuestos
        x0,y0,x1,y1 = to_px(COL_RUT)
        draw.rectangle([x0,y0,x1,y1], outline=(255,0,0), width=3)     # rojo = RUT/P
        x0,y0,x1,y1 = to_px(COL_PART)
        draw.rectangle([x0,y0,x1,y1], outline=(0,128,255), width=3)   # azul = Participación

        # Marcar x-centers detectados de headers
        for xc in rut_xs:
            draw.line([(xc, 0), (xc, H)], fill=(255,0,0), width=1)
        for xc in part_xs:
            draw.line([(xc, 0), (xc, H)], fill=(0,128,255), width=1)

        out_name = OVERLAY_DIR / f"{img_path.parent.name}_{img_path.name}_overlay.png"
        im2.save(out_name)

    return {
        "doc": img_path.parent.name,
        "img": img_path.name,
        "W": W,
        "H": H,
        "rut_hdr_hits": len(rut_xs),
        "part_hdr_hits": len(part_xs),
        "COL_RUT": COL_RUT,
        "COL_PART": COL_PART,
    }

# Ejecutar diagnóstico sobre las subcarpetas de imágenes
rows = []
for sub in sorted([p for p in folder_img.iterdir() if p.is_dir()]):
    imgs = sorted(list(sub.glob("*.png")) + list(sub.glob("*.jpg")) + list(sub.glob("*.jpeg")))
    if MAX_IMGS_PER_DOC is not None:
        imgs = imgs[:MAX_IMGS_PER_DOC]
    for img in imgs:
        rows.append(probe_headers(img))

diag = pd.DataFrame(rows)
print(f"Imágenes diagnosticadas: {len(diag)}")
display(diag.head(20))

# Resumen por documento
if not diag.empty:
    summary = (diag
               .groupby("doc", as_index=False)
               .agg(rut_hits_total=("rut_hdr_hits","sum"),
                    part_hits_total=("part_hdr_hits","sum"),
                    imgs=("img","count"),
                    rut_cols=("COL_RUT", lambda x: x.iloc[0]),
                    part_cols=("COL_PART", lambda x: x.iloc[0])))
    print("Resumen por documento:")
    display(summary)
    print(f"Overlays guardados en: {OVERLAY_DIR}")
else:
    print("No se diagnosticó ninguna imagen.")


Imágenes diagnosticadas: 12


,doc,img,W,H,rut_hdr_hits,part_hdr_hits,COL_RUT,COL_PART
0,70618760K,70618760K_p0005.png,1820,2573,0,0,"(0.14, 0.28)","(0.73, 0.8)"
1,737543456,737543456_p0015.png,1820,2573,0,0,"(0.14, 0.28)","(0.73, 0.8)"
2,761234589,761234589_p0008.png,1820,2573,0,0,"(0.14, 0.28)","(0.73, 0.8)"
3,763456476,763456476_p0002.png,1820,2573,0,2,"(0.14, 0.28)","(0.6153846153846154, 0.7357142857142858)"
4,77123458K,77123458K_p0008.png,1870,2420,0,1,"(0.14, 0.28)","(0.73, 0.8)"
5,778765477,778765477_p0006.png,1820,2573,0,2,"(0.14, 0.28)","(0.6192307692307693, 0.739010989010989)"
6,778765522,778765522_p0012.png,1820,2573,0,2,"(0.14, 0.28)","(0.6214285714285714, 0.7412087912087912)"
7,789654322,789654322_p0001.png,1820,2573,0,0,"(0.14, 0.28)","(0.73, 0.8)"
8,887655999,887655999_p0005.png,1820,2573,0,0,"(0.14, 0.28)","(0.73, 0.8)"
9,987654321,987654321_p0008.png,1820,2573,0,1,"(0.14, 0.28)","(0.6071428571428571, 0.7269230769230769)"


Resumen por documento:


,doc,rut_hits_total,part_hits_total,imgs,rut_cols,part_cols
0,70618760K,0,0,1,"(0.14, 0.28)","(0.73, 0.8)"
1,737543456,0,0,1,"(0.14, 0.28)","(0.73, 0.8)"
2,761234589,0,0,1,"(0.14, 0.28)","(0.73, 0.8)"
3,763456476,0,2,1,"(0.14, 0.28)","(0.6153846153846154, 0.7357142857142858)"
4,77123458K,0,1,1,"(0.14, 0.28)","(0.73, 0.8)"
5,778765477,0,2,1,"(0.14, 0.28)","(0.6192307692307693, 0.739010989010989)"
6,778765522,0,2,1,"(0.14, 0.28)","(0.6214285714285714, 0.7412087912087912)"
7,789654322,0,0,1,"(0.14, 0.28)","(0.73, 0.8)"
8,887655999,0,0,1,"(0.14, 0.28)","(0.73, 0.8)"
9,987654321,0,1,1,"(0.14, 0.28)","(0.6071428571428571, 0.7269230769230769)"


Overlays guardados en: /content/_debug_headers


In [14]:
# @title
# ========= PARCHE: autocalibrar columnas por encabezados (RUT / Pasaporte) y (Participación ≥ 10%) =========
import re, unicodedata, numpy as np
from PIL import Image, ImageOps
import pytesseract

# Defaults (si no encontramos encabezados usamos esto)
ROW_TOP, ROW_BOTTOM = 0.33, 0.92       # banda vertical del cuerpo de la tabla
COL_RUT_DEF  = (0.14, 0.28)            # 2ª columna
COL_PART_DEF = (0.73, 0.80)            # antepenúltima

HDR_PAT_RUT  = re.compile(r"rut\s*/?\s*pasaporte", re.I)
HDR_PAT_PART = re.compile(r"participaci[oó]n", re.I)

def _ocr_data_full(pil, psm="6"):
    df = pytesseract.image_to_data(pil, lang="spa", config=f"--psm {psm}", output_type=pytesseract.Output.DATAFRAME)
    df = df.dropna(subset=["text","conf"])
    df["text"] = df["text"].astype(str).str.strip()
    df = df[(df["text"]!="") & (df["conf"].astype(float) > 40)]
    return df

def autocalibrate_columns(pil: Image.Image):
    """
    Devuelve (COL_RUT, COL_PART) en coordenadas relativas [0..1] para esta imagen,
    detectando encabezados por OCR. Si no se detecta, vuelve a los defaults.
    """
    W, H = pil.size
    df = _ocr_data_full(ImageOps.autocontrast(pil), psm="4")
    if df.empty:
        return COL_RUT_DEF, COL_PART_DEF

    # Tomamos el x-centro de las palabras que hacen match con cada encabezado
    rut_xs, part_xs = [], []
    for _, r in df.iterrows():
        t = r["text"]
        if HDR_PAT_RUT.search(t):
            rut_xs.append(r["left"] + r["width"]/2)
        elif HDR_PAT_PART.search(t):
            part_xs.append(r["left"] + r["width"]/2)

    if not rut_xs and not part_xs:
        return COL_RUT_DEF, COL_PART_DEF

    # Construimos columnas en torno a los headers encontrados
    # ancho de columna = ~12% del ancho (ajustable)
    col_w = 0.12 * W

    def make_band(xc):
        x0 = max(0, int(xc - col_w*0.55))
        x1 = min(W, int(xc + col_w*0.45))
        return (x0/W, ROW_TOP, x1/W, ROW_BOTTOM)  # (x0r,y0r,x1r,y1r)

    COL_RUT = COL_RUT_DEF
    COL_PART = COL_PART_DEF

    if rut_xs:
        xc = float(np.median(rut_xs))
        x0r, y0r, x1r, y1r = make_band(xc)
        COL_RUT = (x0r, x1r)
    if part_xs:
        xc = float(np.median(part_xs))
        x0r, y0r, x1r, y1r = make_band(xc)
        COL_PART = (x0r, x1r)

    return COL_RUT, COL_PART

# Helpers de recorte y data (si no los tienes ya en tu notebook)
def _crop_rel(pil, x0r, y0r, x1r, y1r):
    W, H = pil.size
    box = (int(W*x0r), int(H*y0r), int(W*x1r), int(H*y1r))
    return pil.crop(box), box

def _ocr_data(img_pil, psm="6"):
    df = pytesseract.image_to_data(img_pil, lang="spa", config=f"--psm {psm}", output_type=pytesseract.Output.DATAFRAME)
    df = df.dropna(subset=["text"])
    df["text"] = df["text"].astype(str).str.strip()
    return df[df["text"]!=""]


no dio ningun resultado, al parecer por el ajuste de lectura de las columnas

mal calibrado, tomo rut y le inserto texto a la tabla

In [7]:
# @title
# ====================== PASO 4 — EXTRACCIÓN DESDE IMÁGENES (ROBUSTO + IA) ======================
# Trabaja con las imágenes generadas en el Paso 3 dentro de: folder_img / <RUT_PJ> / <RUT_PJ>_p####.png

# Instalar dependencias si falta algo
try:
    import easyocr  # IA OCR
except Exception:
    !pip -q install easyocr opencv-python-headless

import re, unicodedata, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image
import pytesseract, cv2  #OCR pytesseract
import easyocr  #OCR extraer texto de imagenes de manera simple y eficiente

# ====== Config ======
# Usa las mismas rutas que definiste en Partes 1–3
# folder_img = Path("/content/pdf_pages_benef_finales")

PRINT_MATCH_SUMMARY = True   # imprime un breve log por imagen
SAVE_DEBUG_TXT      = False  # guarda un .txt al lado de la imagen con el OCR usado

# ====== Inicializar EasyOCR (IA); si no hay GPU, usará CPU (más lento) ======
easy_reader = easyocr.Reader(['es','en'], gpu=True)

# ====== Utilidades ======
def norm(s: str) -> str:
    s = s or ""
    s = s.lower()
    s = unicodedata.normalize("NFD", s)
    return "".join(ch for ch in s if unicodedata.category(ch) != "Mn")

def normalize_rut_11_keep_k(s: str) -> str:
    """Mantiene K, elimina separadores y rellena a 11 chars por la izquierda."""
    s = re.sub(r"[^0-9Kk]", "", s or "")
    if not s:
        return ""
    s = s.replace("k","K")
    return s.rjust(11, "0")[:11]

def parse_percent_cell(val: str):
    if val is None:
        return None, ""
    raw = str(val).strip()
    txt = raw.replace(" ", "").replace(",", ".").replace("%", "")
    txt = re.sub(r"[^0-9.]", "", txt)
    if txt == "":
        return None, raw
    try:
        return float(txt), raw
    except ValueError:
        return None, raw

def page_number_from_name(name: str):
    m = re.search(r"_p(\d{4})\.(?:png|jpg|jpeg)$", name, flags=re.I)
    return int(m.group(1)) if m else None

# ====== Preprocesamiento (OpenCV) para mejorar OCR ======
def preprocess_variants(pil_img: Image.Image):
    """Genera varias versiones de la imagen para OCR: original, gris, escalada, binarizada, morfología."""
    img = np.array(pil_img.convert("RGB"))
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    variants = []

    def add(name, arr):
        variants.append((name, Image.fromarray(arr) if arr.ndim==2 else Image.fromarray(arr)))

    # 0) Original
    add("rgb", img)
    # 1) Gris
    add("gray", gray)
    # 2) Escala 1.5x (mejora letras pequeñas)
    gray_big = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    add("gray_x1.5", gray_big)
    # 3) Otsu
    _, otsu = cv2.threshold(gray_big, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    add("otsu", otsu)
    # 4) Adaptativa
    adap = cv2.adaptiveThreshold(gray_big, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 31, 15)
    add("adapt_mean", adap)
    # 5) Morfología ligera (cerrar huecos)
    kernel = np.ones((2,2), np.uint8)
    morph = cv2.morphologyEx(otsu, cv2.MORPH_CLOSE, kernel, iterations=1)
    add("morph_close", morph)
    # 6) Bilateral + Otsu (reduce ruido)
    bil = cv2.bilateralFilter(gray_big, 7, 50, 50)
    _, otsu2 = cv2.threshold(bil, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    add("bil_otsu", otsu2)

    return variants

# ====== OCR híbrido: Tesseract (varios PSM) → EasyOCR (IA) ======
def ocr_text_best(pil_img: Image.Image):
    """
    Prueba varias variantes/preprocesos + Tesseract con distintos PSM.
    Si el texto es corto, usa EasyOCR (IA). Devuelve (texto, etiqueta_metodo).
    """
    variants = preprocess_variants(pil_img)
    psm_list = ["6", "4", "11", "3"]  # 6: bloques, 4: columna, 11: sparse, 3: auto
    best_txt, best_src, best_len = "", "", 0

    for name, img in variants:
        for psm in psm_list:
            try:
                txt = pytesseract.image_to_string(img, lang="spa", config=f"--psm {psm}")
            except Exception:
                txt = ""
            tl = len(txt.strip())
            if tl > best_len:
                best_txt, best_src, best_len = txt, f"tesseract:{name}:psm{psm}", tl

    # respaldo IA
    if best_len < 25:
        try:
            arr = np.array(dict(variants)["gray_x1.5"])
            chunks = easy_reader.readtext(arr, detail=0, paragraph=True)
            txt_easy = "\n".join(chunks)
            if len(txt_easy.strip()) > best_len:
                return txt_easy, "easyocr:gray_x1.5"
        except Exception:
            pass

    return best_txt, best_src or "tesseract"

# ====== Palabras clave de sección (normalizadas) ======
SECTION_HINTS = tuple(map(norm, [
    "Identificación de Beneficiarios Finales",
    "Identificacion de Beneficiarios Finales",
    "Identificación de Beneficiario Final",
    "Identificacion de Beneficiario Final",
    "beneficiarios finales",
    "identificacion de los beneficiarios finales",
]))

# ====== Patrones para filas (una fila por beneficiario) ======
# RUT flexible (con/ sin puntos y guion, DV K) o Pasaporte alfanumérico (>=6)
RUT_FLEX = r"\d{1,2}\.?\d{3}\.?\d{3}-?[0-9Kk]"
ID_ANY   = rf"(?:{RUT_FLEX}|[0-9A-Za-z]{{6,}})"
PCT_ANY  = r"\d{1,3}(?:[.,]\d+)?%?"

# 1) Por línea (rápido)
ROW_LINE = re.compile(
    rf"^\s*(CI|P)\s+({ID_ANY}).*?({PCT_ANY})\s*$",
    flags=re.IGNORECASE
)
# 2) Multilínea (tolerante a saltos entre ID y %)
ROW_MULTI = re.compile(
    rf"(?:^|\n|\r)\s*(CI|P)\b.*?({ID_ANY}).{{0,220}}?({PCT_ANY})",
    flags=re.IGNORECASE | re.DOTALL
)
# ====== Extracción principal desde IMÁGENES ======
def run_extraction_from_images(img_root: Path) -> pd.DataFrame:
    records = []

    for sub in sorted([p for p in img_root.iterdir() if p.is_dir()]):
        rut_base = sub.name                       # nombre de carpeta = nombre del PDF original
        rut_pj = normalize_rut_11_keep_k(rut_base)

        imgs = sorted(list(sub.glob("*.png")) + list(sub.glob("*.jpg")) + list(sub.glob("*.jpeg")))
        if not imgs:
            records.append({
                "Archivo": f"{rut_base}.pdf","Pagina": "","Rut_PJ": rut_pj,
                "Tipo_doc": "","Rut_bf": "","Rut_bf_num11": "","porc_part_raw": "","porc_part": "",
                "Estado": "Incidencia: no hay imágenes detectadas para este documento.",
            })
            continue

        any_success = False

        for img_path in imgs:
            try:
                pil = Image.open(img_path).convert("RGB")
            except Exception:
                continue

            text, method = ocr_text_best(pil)
            page_num = page_number_from_name(img_path.name)
            text_norm = norm(text)

            has_section = any(h in text_norm for h in SECTION_HINTS)

            # ---- 1) matches por LÍNEA
            rows_local = []
            for ln in text.splitlines():
                m = ROW_LINE.search(ln)
                if not m:
                    continue
                tipo = m.group(1).upper()
                doc_id = m.group(2).strip()
                part   = m.group(3).strip()
                rows_local.append({"tipo": tipo, "id": doc_id, "part": part})

            # ---- 2) si pocas filas, intenta MULTILÍNEA sobre todo el texto
            if len(rows_local) <= 1:
                for m in ROW_MULTI.finditer(text):
                    tipo = m.group(1).upper()
                    doc_id = m.group(2).strip()
                    part   = m.group(3).strip()
                    rows_local.append({"tipo": tipo, "id": doc_id, "part": part})

            if PRINT_MATCH_SUMMARY:
                print(f"{img_path.name} -> metodo={method} section={has_section} matches={len(rows_local)}")

            if SAVE_DEBUG_TXT:
                with open(img_path.with_suffix(".txt"), "w", encoding="utf-8") as f:
                    f.write(f"METHOD={method}\n")
                    f.write(text)

            if rows_local:
                any_success = True
                for row in rows_local:
                    p_val, p_raw = parse_percent_cell(row["part"])
                    records.append({
                        "Archivo": f"{rut_base}.pdf",
                        "Pagina": page_num if page_num is not None else "",
                        "Rut_PJ": rut_pj,
                        "Tipo_doc": row["tipo"],                           # CI o P
                        "Rut_bf": row["id"],
                        "Rut_bf_num11": normalize_rut_11_keep_k(row["id"]) if row["tipo"] == "CI" else "",
                        "porc_part_raw": p_raw,
                        "porc_part": p_val,
                        "Estado": "Success",
                    })
            else:
                # Registrar incidencia por página
                records.append({
                    "Archivo": f"{rut_base}.pdf",
                    "Pagina": page_num if page_num is not None else "",
                    "Rut_PJ": rut_pj,
                    "Tipo_doc": "",
                    "Rut_bf": "",
                    "Rut_bf_num11": "",
                    "porc_part_raw": "",
                    "porc_part": "",
                    "Estado": ("Incidencia: página detectada sin filas legibles de la tabla."
                               if has_section else
                               "Incidencia: no se detectó claramente la sección en esta imagen."),
                })

        # si no hubo ninguna fila exitosa, ya quedaron incidencias por página; no agregamos extra por doc

    cols = ["Archivo","Pagina","Rut_PJ","Tipo_doc","Rut_bf","Rut_bf_num11","porc_part_raw","porc_part","Estado"]
    return pd.DataFrame(records, columns=cols)

# ====================== Ejecutar y visualizar (sin CSV) ======================
df = run_extraction_from_images(folder_img)
print(f"Filas obtenidas desde IMÁGENES: {len(df)}")
display(df.head(30))

# ====================== Histórico en memoria (opcional) ======================
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

try:
    hist_bf
except NameError:
    hist_bf = pd.DataFrame(columns=df.columns)

dedup_keys = ["Archivo", "Pagina", "Rut_PJ", "Tipo_doc", "Rut_bf", "porc_part_raw"]
hist_bf = pd.concat([hist_bf, df.astype(str)], ignore_index=True)
hist_bf.drop_duplicates(subset=dedup_keys, keep="last", inplace=True)

marker = {col: "" for col in hist_bf.columns}
marker["Archivo"] = "data_date_part"
marker["Estado"]  = timestamp
hist_bf = pd.concat([hist_bf, pd.DataFrame([marker])], ignore_index=True)

display(hist_bf.tail(20))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 13.4 MB/s eta 0:00:00


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete70618760K_p0005.png -> metodo=tesseract:rgb:psm6 section=True matches=0
737543456_p0015.png -> metodo=tesseract:morph_close:psm11 section=True matches=0
761234589_p0008.png -> metodo=tesseract:gray_x1.5:psm11 section=True matches=1
763456476_p0002.png -> metodo=tesseract:morph_close:psm11 section=True matches=2
77123458K_p0008.png -> metodo=tesseract:bil_otsu:psm6 section=True matches=0
778765477_p0006.png -> metodo=tesseract:gray_x1.5:psm6 section=True matches=0
778765522_p0012.png -> metodo=tesseract:morph_close:psm11 section=True matches=0
789654322_p0001.png -> metodo=tesseract:bil_otsu:psm11 section=True matches=1
887655999_p0005.png -> metodo=tesseract:morph_close:psm6 section=True matches=0
987654321_p0008.png -> metodo=tesseract:morph_close:psm6 section=True matches=0
998765437_p0007.png -> metodo=tesseract:morph_close:psm6 section=True matches=0
998765477_p0009.png -> metodo=tesseract:morph_close:psm

,Archivo,Pagina,Rut_PJ,Tipo_doc,Rut_bf,Rut_bf_num11,porc_part_raw,porc_part,Estado
0,70618760K.pdf,5,0070618760K,,,,,,Incidencia: página detectada sin filas legible...
1,737543456.pdf,15,00737543456,,,,,,Incidencia: página detectada sin filas legible...
2,761234589.pdf,8,00761234589,CI,15123456-K,0015123456K,02,2.0,Success
3,763456476.pdf,2,00763456476,P,tineczoricvalla,,4,4.0,Success
4,763456476.pdf,2,00763456476,CI,6453627-8,00064536278,100,100.0,Success
5,77123458K.pdf,8,0077123458K,,,,,,Incidencia: página detectada sin filas legible...
6,778765477.pdf,6,00778765477,,,,,,Incidencia: página detectada sin filas legible...
7,778765522.pdf,12,00778765522,,,,,,Incidencia: página detectada sin filas legible...
8,789654322.pdf,1,00789654322,CI,78987651,00078987651,010,10.0,Success
9,887655999.pdf,5,00887655999,,,,,,Incidencia: página detectada sin filas legible...


,Archivo,Pagina,Rut_PJ,Tipo_doc,Rut_bf,Rut_bf_num11,porc_part_raw,porc_part,Estado
0,70618760K.pdf,5,0070618760K,,,,,,Incidencia: página detectada sin filas legible...
1,737543456.pdf,15,00737543456,,,,,,Incidencia: página detectada sin filas legible...
2,761234589.pdf,8,00761234589,CI,15123456-K,0015123456K,02,2.0,Success
3,763456476.pdf,2,00763456476,P,tineczoricvalla,,4,4.0,Success
4,763456476.pdf,2,00763456476,CI,6453627-8,00064536278,100,100.0,Success
5,77123458K.pdf,8,0077123458K,,,,,,Incidencia: página detectada sin filas legible...
6,778765477.pdf,6,00778765477,,,,,,Incidencia: página detectada sin filas legible...
7,778765522.pdf,12,00778765522,,,,,,Incidencia: página detectada sin filas legible...
8,789654322.pdf,1,00789654322,CI,78987651,00078987651,010,10.0,Success
9,887655999.pdf,5,00887655999,,,,,,Incidencia: página detectada sin filas legible...


### Aquí abajito los que no funcionaron en extraccion de datos de los pdf/imagenes

no funciono, pero casiii

In [41]:
# @title
# Parte 4 (imágenes): OCR Tesseract + EasyOCR (IA) sobre /pdf_pages_benef_finales
import re, unicodedata, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image
import pytesseract

# Reusar variables de Partes previas
# folder_img = Path("/content/pdf_pages_benef_finales")

# === EasyOCR (IA) ===
try:
    import easyocr
    easy_reader = easyocr.Reader(['es','en'], gpu=True)
    EASY_OK = True
except Exception as e:
    print("EasyOCR no disponible (usa solo Tesseract). Para habilitar IA: !pip -q install easyocr")
    EASY_OK = False

def ocr_tesseract(img: Image.Image) -> str:
    return pytesseract.image_to_string(img, lang="spa", config="--psm 6")

def ocr_easy(img: Image.Image) -> str:
    if not EASY_OK:
        return ""
    arr = np.array(img)

    chunks = easy_reader.readtext(arr, detail=0, paragraph=True) # detail=0 -> solo texto, paragraph=True agrupa mejor
    return "\n".join(chunks)

def ocr_smart(img: Image.Image) -> str:
    txt = ocr_tesseract(img)
    if len(txt.strip()) < 12:  # si tesseract sacó muy poco, intentamos IA
        txt2 = ocr_easy(img)
        if len(txt2.strip()) > len(txt.strip()):
            return txt2
    return txt

def normalize_rut_11_keep_k(s: str) -> str:
    s = re.sub(r"[^0-9Kk]", "", s or "")
    if not s:
        return ""
    s = s.replace("k","K")
    return s.rjust(11, "0")[:11]  # mantiene K

def parse_percent_cell(val: str):
    if val is None:
        return None, ""
    raw = str(val).strip()
    txt = raw.replace(" ", "").replace(",", ".").replace("%", "")
    txt = re.sub(r"[^0-9.]", "", txt)
    if txt == "":
        return None, raw
    try:
        return float(txt), raw
    except ValueError:
        return None, raw

def norm(s: str) -> str:
    s = s or ""
    s = s.lower()
    s = unicodedata.normalize("NFD", s)
    return "".join(ch for ch in s if unicodedata.category(ch) != "Mn")

SECTION_HINTS = tuple(map(norm, [
    "Identificación de Beneficiarios Finales",
    "Identificacion de Beneficiarios Finales",
    "Identificación de Beneficiario Final",
    "Identificacion de Beneficiario Final"
]))

# Patrón de fila: CI|P + id (alfa/num/guion) + porcentaje (con o sin %)
ROW_REGEX = re.compile(r"\b(CI|P)\b.*?([0-9A-Za-z\-]{6,}).{0,120}?(\d{1,3}(?:[.,]\d+)?%?)\b")

def page_number_from_name(name: str) -> int | None:
    # espera sufijo _p####.png
    m = re.search(r"_p(\d{4})\.(?:png|jpg|jpeg)$", name, flags=re.I)
    return int(m.group(1)) if m else None

def run_extraction_from_images(img_root: Path) -> pd.DataFrame:
    records = []
    # Recorre cada carpeta de un PDF (rut)
    for sub in sorted([p for p in img_root.iterdir() if p.is_dir()]):
        rut_base = sub.name  # nombre de carpeta = RUT_PJ del PDF original
        rut_pj = normalize_rut_11_keep_k(rut_base)

        # Todas las imágenes detectadas para ese PDF
        imgs = sorted(list(sub.glob("*.png")) + list(sub.glob("*.jpg")) + list(sub.glob("*.jpeg")))
        if not imgs:
            records.append({
                "Archivo": f"{rut_base}.pdf",
                "Pagina": "",
                "Rut_PJ": rut_pj,
                "Tipo_doc": "",
                "Rut_bf": "",
                "Rut_bf_num11": "",
                "porc_part_raw": "",
                "porc_part": "",
                "Estado": "Incidencia: no hay imágenes detectadas para este documento.",
            })
            continue

        found_any = False
        for img_path in imgs:
            try:
                img = Image.open(img_path).convert("RGB")
            except Exception:
                continue

            txt = ocr_smart(img)
            txt_norm = norm(txt)

            # ¿aparece la sección?
            has_section = any(h in txt_norm for h in SECTION_HINTS)

            # Buscar filas en todas las líneas (regex tolerante)
            rows_local = []
            for ln in txt.splitlines():
                ln = ln.strip()
                if not ln:
                    continue
                m = ROW_REGEX.search(ln)
                if m:
                    tipo = m.group(1).upper()
                    doc_id = m.group(2).strip()
                    part_str = m.group(3).strip()
                    rows_local.append({"tipo": tipo, "id": doc_id, "part": part_str})

            page_num = page_number_from_name(img_path.name)

            if rows_local:
                found_any = True
                for row in rows_local:
                    p_val, p_raw = parse_percent_cell(row["part"])
                    records.append({
                        "Archivo": f"{rut_base}.pdf",
                        "Pagina": page_num if page_num is not None else "",
                        "Rut_PJ": rut_pj,
                        "Tipo_doc": row["tipo"],  # CI o P
                        "Rut_bf": row["id"],
                        "Rut_bf_num11": normalize_rut_11_keep_k(row["id"]) if row["tipo"] == "CI" else "",
                        "porc_part_raw": p_raw,
                        "porc_part": p_val,
                        "Estado": "Success",
                    })
            else:
                # sin filas en esa imagen; registramos incidencia a nivel de página
                records.append({
                    "Archivo": f"{rut_base}.pdf",
                    "Pagina": page_num if page_num is not None else "",
                    "Rut_PJ": rut_pj,
                    "Tipo_doc": "",
                    "Rut_bf": "",
                    "Rut_bf_num11": "",
                    "porc_part_raw": "",
                    "porc_part": "",
                    "Estado": ("Incidencia: página detectada sin filas legibles de la tabla."
                               if has_section else
                               "Incidencia: no se detectó claramente la sección en esta imagen."),
                })

        if not found_any:
            # si ninguna imagen produjo filas, ya quedaron incidencias por página;
            # no añadimos una extra a nivel de documento para no duplicar.
            pass

    cols = ["Archivo","Pagina","Rut_PJ","Tipo_doc","Rut_bf","Rut_bf_num11","porc_part_raw","porc_part","Estado"]
    return pd.DataFrame(records, columns=cols)

# === Ejecutar y visualizar (sin CSV) ===
df = run_extraction_from_images(folder_img)
print(f"Filas obtenidas desde IMÁGENES: {len(df)}")
display(df.head(30))

# === Histórico en memoria (igual que antes, sin exportar) ===
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

try:
    hist_bf
except NameError:
    hist_bf = pd.DataFrame(columns=df.columns)

dedup_keys = ["Archivo", "Pagina", "Rut_PJ", "Tipo_doc", "Rut_bf", "porc_part_raw"]
hist_bf = pd.concat([hist_bf, df.astype(str)], ignore_index=True)
hist_bf.drop_duplicates(subset=dedup_keys, keep="last", inplace=True)

marker = {col: "" for col in hist_bf.columns}
marker["Archivo"] = "data_date_part"
marker["Estado"]  = timestamp
hist_bf = pd.concat([hist_bf, pd.DataFrame([marker])], ignore_index=True)

display(hist_bf.tail(20))


Filas obtenidas desde IMÁGENES: 12


,Archivo,Pagina,Rut_PJ,Tipo_doc,Rut_bf,Rut_bf_num11,porc_part_raw,porc_part,Estado
0,70618760K.pdf,5,0070618760K,,,,,,Incidencia: página detectada sin filas legible...
1,737543456.pdf,15,00737543456,,,,,,Incidencia: página detectada sin filas legible...
2,761234589.pdf,8,00761234589,,,,,,Incidencia: página detectada sin filas legible...
3,763456476.pdf,2,00763456476,,,,,,Incidencia: página detectada sin filas legible...
4,77123458K.pdf,8,0077123458K,,,,,,Incidencia: página detectada sin filas legible...
5,778765477.pdf,6,00778765477,,,,,,Incidencia: página detectada sin filas legible...
6,778765522.pdf,12,00778765522,,,,,,Incidencia: página detectada sin filas legible...
7,789654322.pdf,1,00789654322,CI,126454987,00126454987,"15,02",15.02,Success
8,887655999.pdf,5,00887655999,,,,,,Incidencia: página detectada sin filas legible...
9,987654321.pdf,8,00987654321,,,,,,Incidencia: página detectada sin filas legible...


,Archivo,Pagina,Rut_PJ,Tipo_doc,Rut_bf,Rut_bf_num11,porc_part_raw,porc_part,Estado
0,70618760K.pdf,5,0070618760K,,,,,,Incidencia: página detectada sin filas legible...
1,737543456.pdf,15,00737543456,,,,,,Incidencia: página detectada sin filas legible...
2,761234589.pdf,8,00761234589,,,,,,Incidencia: página detectada sin filas legible...
3,763456476.pdf,2,00763456476,,,,,,Incidencia: página detectada sin filas legible...
4,77123458K.pdf,8,0077123458K,,,,,,Incidencia: página detectada sin filas legible...
5,778765477.pdf,6,00778765477,,,,,,Incidencia: página detectada sin filas legible...
6,778765522.pdf,12,00778765522,,,,,,Incidencia: página detectada sin filas legible...
7,789654322.pdf,1,00789654322,CI,126454987,00126454987,"15,02",15.02,Success
8,887655999.pdf,5,00887655999,,,,,,Incidencia: página detectada sin filas legible...
9,987654321.pdf,8,00987654321,,,,,,Incidencia: página detectada sin filas legible...


no funciono :(

In [39]:
# @title
# Parte 4 (OCR): extraer filas de la tabla y generar CSV con columna Estado
import pandas as pd
import fitz, re, unicodedata, pytesseract ##ocr
from PIL import Image
from pathlib import Path

#rut en formato penumdoc
def normalize_rut_11_keep_k(s: str) -> str:
    s = re.sub(r"[^0-9Kk]", "", s or "")
    if not s:
        return ""
    s = s.replace("k","K")
    return s.rjust(11, "0")[:11]  # respeta K

def normalize_company_rut_from_filename(fname: str) -> str:
    return normalize_rut_11_keep_k(Path(fname).stem)

def parse_percent_cell(val: str):
    if val is None:
        return None, ""
    raw = str(val).strip()
    txt = raw.replace(" ", "").replace(",", ".").replace("%", "")
    txt = re.sub(r"[^0-9.]", "", txt)
    if txt == "":
        return None, raw
    try:
        return float(txt), raw
    except ValueError:
        return None, raw

def norm(s: str) -> str:
    s = s or ""
    s = s.lower()
    s = unicodedata.normalize("NFD", s)
    return "".join(ch for ch in s if unicodedata.category(ch) != "Mn")

#volvemos a identificar las secciones relevantes
SECTION_HINTS = tuple(map(norm, [
    "Identificación de Beneficiarios Finales",
    "Identificacion de Beneficiarios Finales",
    "Identificación de Beneficiario Final",
    "Identificacion de Beneficiario Final"
]))

##detectamos las palabras claves
COL_HINTS = tuple(map(norm, [
    "RUT/Pasaporte","RUT","R.U.T","Pasaporte",
    "Nombre Completo","Nacionalidad","Fecha de Nacimiento",
    "Domicilio, Ciudad y País","Participación","Participacion"
]))

DPI  = 220 ##mejor nitidez
zoom = DPI/72
MAT  = fitz.Matrix(zoom, zoom)

def page_text_pdf(doc, i) -> str:
    return doc.load_page(i).get_text("text") or ""

def page_text_ocr(doc, i) -> str:
    pix = doc.load_page(i).get_pixmap(matrix=MAT, alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    return pytesseract.image_to_string(img, lang="spa")

def score_text(txt_norm: str) -> int:
    score = 0
    if any(h in txt_norm for h in SECTION_HINTS):
        score += 5
    for c in COL_HINTS:
        if c in txt_norm:
            score += 1
    return score

def select_best_page(doc):
    best_idx, best_score = -1, -1
    for i in range(doc.page_count):
        t_pdf = page_text_pdf(doc, i)
        s_pdf = score_text(norm(t_pdf))
        if s_pdf < 5:
            t_ocr = page_text_ocr(doc, i)
            s_pdf = max(s_pdf, score_text(norm(t_ocr)))
        if s_pdf > best_score:
            best_idx, best_score = i, s_pdf
    return best_idx if best_idx >= 0 and best_score > 0 else None

def extract_rows_from_page(page: fitz.Page):
    rows = []
    # 1) Intentar con detector de tablas de PyMuPDF
    try:
        tables = page.find_tables()
        if tables and tables.tables:
            tbl = max(tables.tables, key=lambda t: (t.bbox[2]-t.bbox[0])*(t.bbox[3]-t.bbox[1]))
            data = tbl.extract()
            start = 2 if len(data) > 2 else 0
            for r in data[start:]:
                tipo = (r[0] or "").strip().upper()
                doc_id = (r[1] or "").strip()
                part   = (r[6] or "").strip() if len(r) > 6 else ""
                if not any([tipo, doc_id, part]):
                    continue
                rows.append({"tipo": tipo, "id": doc_id, "part": part})
            if rows:
                return rows
    except Exception:
        pass

    # 2) Respaldo OCR: leer línea por línea y buscar patrón CI/P + id + porcentaje
    #    (esto soporta páginas que son solo imagen/escaneo)
    pix = page.get_pixmap(matrix=MAT, alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    text_ocr = pytesseract.image_to_string(img, lang="spa")
    for ln in text_ocr.splitlines():
        ln = ln.strip()
        if not ln:
            continue
        m = re.search(r"\b(CI|P)\b\s+([A-Za-z0-9\-]+).{0,120}?(\d{1,3}(?:[.,]\d+)?%?)\b", ln)
        if m:
            rows.append({"tipo": m.group(1).upper(), "id": m.group(2).strip(), "part": m.group(3).strip()})
    return rows

def status_row(pdf_name, rut_pj, page_num, msg):
    return {
        "Archivo": pdf_name,
        "Pagina": page_num if page_num is not None else "",
        "Rut_PJ": rut_pj,
        "Tipo_doc": "",
        "Rut_bf": "",
        "Rut_bf_num11": "",
        "porc_part_raw": "",
        "porc_part": "",
        "Estado": msg,
    }

def run_extraction_to_csv(input_dir: Path, out_csv: Path):
    records = []
    for pdf in sorted(input_dir.glob("*.pdf")):
        rut_pj = normalize_company_rut_from_filename(pdf.name)
        with fitz.open(pdf) as doc:
            best = select_best_page(doc)
            if best is None:
                records.append(status_row(pdf.name, rut_pj, None,
                    "Incidencia: no se encontró la sección 'Identificación de Beneficiarios Finales'."))
                continue
            page = doc.load_page(best)
            rows = extract_rows_from_page(page)
            if not rows:
                records.append(status_row(pdf.name, rut_pj, best+1,
                    "Incidencia: se encontró la sección pero no se pudo leer información de la tabla."))
                continue
            for row in rows:
                tipo = (row.get("tipo") or "").strip().upper()
                id_text = (row.get("id") or "").strip()
                p_val, p_raw = parse_percent_cell(row.get("part"))
                records.append({
                    "Archivo": pdf.name,
                    "Pagina": best + 1,
                    "Rut_PJ": rut_pj,                                     # 11 (con K si existe)
                    "Tipo_doc": "CI" if tipo == "CI" else ("P" if tipo == "P" else ""),
                    "Rut_bf": id_text,
                    "Rut_bf_num11": normalize_rut_11_keep_k(id_text) if tipo == "CI" else "",
                    "porc_part_raw": p_raw,
                    "porc_part": p_val,
                    "Estado": "Success",
                })
    df = pd.DataFrame(records, columns=[
        "Archivo","Pagina","Rut_PJ","Tipo_doc","Rut_bf","Rut_bf_num11","porc_part_raw","porc_part","Estado"
    ])
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    return df

# Ejecutar extracción sobre los PDFs subidos en la Parte 2
input_dir = folder_pdf
out_csv = Path("/content/extraccion_benef_finales.csv")
df = run_extraction_to_csv(input_dir, out_csv)
print(f"CSV generado: {out_csv} ({len(df)} filas)")
df.head(20)


CSV generado: /content/extraccion_benef_finales.csv (12 filas)


,Archivo,Pagina,Rut_PJ,Tipo_doc,Rut_bf,Rut_bf_num11,porc_part_raw,porc_part,Estado
0,70618760K.pdf,5,0070618760K,,,,,,Incidencia: se encontró la sección pero no se ...
1,737543456.pdf,15,00737543456,,,,,,Incidencia: se encontró la sección pero no se ...
2,761234589.pdf,8,00761234589,,,,,,Incidencia: se encontró la sección pero no se ...
3,763456476.pdf,2,00763456476,,,,,,Incidencia: se encontró la sección pero no se ...
4,77123458K.pdf,8,0077123458K,,,,,,Incidencia: se encontró la sección pero no se ...
5,778765477.pdf,6,00778765477,,,,,,Incidencia: se encontró la sección pero no se ...
6,778765522.pdf,12,00778765522,,,,,,Incidencia: se encontró la sección pero no se ...
7,789654322.pdf,1,00789654322,,,,,,Incidencia: se encontró la sección pero no se ...
8,887655999.pdf,5,00887655999,,,,,,Incidencia: se encontró la sección pero no se ...
9,987654321.pdf,8,00987654321,,,,,,Incidencia: se encontró la sección pero no se ...


intento n°2 de OCR para extraer datos rut y % de participacion

In [13]:
# @title
# Parte 4 (IA): extractor híbrido con EasyOCR (IA) + Tesseract de respaldo
import pandas as pd
import fitz, re, unicodedata, pytesseract, easyocr
from PIL import Image
from pathlib import Path

# ========= utilidades =========
def normalize_rut_11_keep_k(s: str) -> str:
    s = re.sub(r"[^0-9Kk]", "", s or "")
    if not s:
        return ""
    s = s.replace("k","K")
    return s.rjust(11, "0")[:11]  # conserva K

def normalize_company_rut_from_filename(fname: str) -> str:
    return normalize_rut_11_keep_k(Path(fname).stem)

def parse_percent_cell(val: str):
    if val is None:
        return None, ""
    raw = str(val).strip()
    txt = raw.replace(" ", "").replace(",", ".").replace("%", "")
    txt = re.sub(r"[^0-9.]", "", txt)
    if txt == "":
        return None, raw
    try:
        return float(txt), raw
    except ValueError:
        return None, raw

def norm(s: str) -> str:
    s = s or ""
    s = s.lower()
    s = unicodedata.normalize("NFD", s)
    return "".join(ch for ch in s if unicodedata.category(ch) != "Mn")

SECTION_HINTS = tuple(map(norm, [
    "Identificación de Beneficiarios Finales",
    "Identificacion de Beneficiarios Finales",
    "Identificación de Beneficiario Final",
    "Identificacion de Beneficiario Final"
]))

# ========= parámetros render =========
DPI  = 220
zoom = DPI/72
MAT  = fitz.Matrix(zoom, zoom)

# ========= OCR IA (EasyOCR) =========
# Cargamos una sola vez (es pesado)
try:
    _easyocr_reader
except NameError:
    _easyocr_reader = easyocr.Reader(['es', 'en'], gpu=False, verbose=False)

def ocr_easy_page_image(page: fitz.Page, rect: fitz.Rect=None) -> str:
    """OCR IA con EasyOCR; full page o recorte."""
    if rect:
        pix = page.get_pixmap(matrix=MAT, clip=rect, alpha=False)
    else:
        pix = page.get_pixmap(matrix=MAT, alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    # EasyOCR devuelve una lista de strings (detail=0). Unimos por saltos.
    lines = _easyocr_reader.readtext(np.array(img), detail=0, paragraph=False)
    return "\n".join(lines)

# Para easyocr: necesitamos numpy
import numpy as np

# ========= helpers detección de página =========
def page_text_pdf(doc, i) -> str:
    return doc.load_page(i).get_text("text") or ""

def page_text_tesseract(page: fitz.Page) -> str:
    pix = page.get_pixmap(matrix=MAT, alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    return pytesseract.image_to_string(img, lang="spa")

def score_text(txt_norm: str) -> int:
    score = 0
    if any(h in txt_norm for h in SECTION_HINTS):
        score += 5
    for kw in ["participacion", "rut", "pasaporte", "nacionalidad", "domicilio"]:
        if kw in txt_norm:
            score += 1
    return score

def select_best_page(doc):
    best_idx, best_score = -1, -1
    for i in range(doc.page_count):
        t = page_text_pdf(doc, i)
        s = score_text(norm(t))
        if s < 5:
            # 1) OCR IA
            try:
                t_ia = ocr_easy_page_image(doc.load_page(i))
                s = max(s, score_text(norm(t_ia)))
            except Exception:
                pass
        if s < 5:
            # 2) Tesseract de respaldo
            try:
                t_ocr = page_text_tesseract(doc.load_page(i))
                s = max(s, score_text(norm(t_ocr)))
            except Exception:
                pass
        if s > best_score:
            best_idx, best_score = i, s
    return best_idx if best_idx >= 0 and best_score > 0 else None

# ========= extractor por palabras (coordenadas) =========
def words_norm(page: fitz.Page):
    W = page.get_text("words")
    out = []
    for x0,y0,x1,y1,txt,*_ in W:
        if txt.strip():
            out.append((x0,y0,x1,y1,txt,norm(txt)))
    return out

def guess_header_y_and_bins(page: fitz.Page):
    # keywords para identificar encabezado inferior
    HEADER_TARGETS = [
        "tipo","rut","pasaporte","nombre","nacionalidad",
        "fecha","nacimiento","domicilio","ciudad","pais",
        "participacion","control","director","direccion","alta"
    ]
    W = words_norm(page)
    if not W:
        return None, None
    cand_y = [y1 for x0,y0,x1,y1,txt,txn in W if any(h in txn for h in HEADER_TARGETS)]
    if not cand_y:
        return None, None
    cand_y.sort()
    header_y = cand_y[int(0.75*len(cand_y))]  # borde inferior aprox encabezado

    # centros de los títulos en la banda del encabezado
    centers = []
    for x0,y0,x1,y1,txt,txn in W:
        if y0 >= header_y-25 and y1 <= header_y+30:
            if any(h in txn for h in HEADER_TARGETS):
                centers.append((x0+x1)/2)
    if not centers:
        # layout de 9 columnas por proporciones
        r = page.rect
        x0, x1 = r.x0+30, r.x1-30
        widths = [14,28,34,20,22,56,28,22,26]
        ratio = sum(widths)
        edges=[x0]; acc=x0
        for w in widths:
            acc += (x1-x0)*(w/ratio); edges.append(acc)
        bins=[(edges[i]+edges[i+1])/2 for i in range(9)]
        return header_y, bins

    centers = sorted(centers)
    bins=[]
    group=[centers[0]]
    for c in centers[1:]:
        if abs(c-group[-1])<25:
            group.append(c)
        else:
            bins.append(sum(group)/len(group)); group=[c]
    bins.append(sum(group)/len(group))
    # Si <7 columnas, usa el fallback de proporciones
    if len(bins)<7:
        r = page.rect
        x0, x1 = r.x0+30, r.x1-30
        widths = [14,28,34,20,22,56,28,22,26]
        ratio = sum(widths)
        edges=[x0]; acc=x0
        for w in widths:
            acc += (x1-x0)*(w/ratio); edges.append(acc)
        bins=[(edges[i]+edges[i+1])/2 for i in range(9)]
    return header_y, bins

def assign_col(xc, bins): return min(range(len(bins)), key=lambda i: abs(xc-bins[i]))

def extract_rows_by_words(page: fitz.Page):
    header_y, bins = guess_header_y_and_bins(page)
    if header_y is None or bins is None:
        return []
    W = words_norm(page)
    body = [(x0,y0,x1,y1,txt,txn) for (x0,y0,x1,y1,txt,txn) in W if y0>header_y+2]
    # agrupar por renglón
    rows=[]; current=[]; cy=None
    for w in sorted(body, key=lambda v:(v[1],v[0])):
        x0,y0,x1,y1,txt,txn = w
        if cy is None or abs(y0-cy)<=6:
            current.append(w); cy = y0 if cy is None else (cy*0.7+y0*0.3)
        else:
            rows.append(current); current=[w]; cy=y0
    if current: rows.append(current)

    out=[]
    for words in rows:
        cells={i:[] for i in range(9)}
        for x0,y0,x1,y1,txt,txn in words:
            col=assign_col((x0+x1)/2, bins)
            cells[col].append((x0,txt))
        for k in cells:
            cells[k]=" ".join(t for _,t in sorted(cells[k], key=lambda p:p[0])).strip()
        tipo=cells.get(0,"").upper()
        doc_id=cells.get(1,"")
        part=cells.get(6,"")
        if not any([tipo,doc_id,part]):
            continue
        m=re.search(r"\b(CI|P)\b", tipo)
        tipo_norm=m.group(1) if m else ("CI" if "CI" in tipo else ("P" if "P" in tipo else ""))
        out.append({"tipo":tipo_norm, "id":doc_id, "part":part})
    return out

# ========= OCR por IA (EasyOCR) y respaldo Tesseract a nivel filas =========
def extract_rows_by_ocr(page: fitz.Page):
    rows=[]
    # 1) IA página completa
    try:
        txt = ocr_easy_page_image(page)
        for ln in txt.splitlines():
            m = re.search(r"\b(CI|P)\b\s+([A-Za-z0-9\-]+).{0,120}?(\d{1,3}(?:[.,]\d+)?%?)\b", ln)
            if m:
                rows.append({"tipo":m.group(1).upper(),"id":m.group(2).strip(),"part":m.group(3).strip()})
    except Exception:
        pass
    if rows:
        return rows

    # 2) IA por columnas recortadas (0:Tipo, 1:ID, 6:%)
    r = page.rect
    widths=[14,28,34,20,22,56,28,22,26]; ratio=sum(widths)
    x0=r.x0+30; x1=r.x1-30
    edges=[x0]; acc=x0
    for w in widths:
        acc += (x1-x0)*(w/ratio); edges.append(acc)
    def clip(i): return fitz.Rect(edges[i], r.y0+150, edges[i+1], r.y1-60)
    try:
        col0 = ocr_easy_page_image(page, clip(0))
        col1 = ocr_easy_page_image(page, clip(1))
        col6 = ocr_easy_page_image(page, clip(6))
        # heurística: primera línea que calce en cada
        def pick(pat, txt):
            for ln in txt.splitlines():
                if re.search(pat, ln.strip()): return ln.strip()
            return ""
        tipo = pick(r"\b(CI|P)\b", col0)
        docid= pick(r"[A-Za-z0-9\-]{6,}", col1)
        part = pick(r"\b\d{1,3}(?:[.,]\d+)?%?\b", col6)
        if any([tipo,docid,part]):
            m=re.search(r"\b(CI|P)\b", tipo.upper())
            tipo_norm=m.group(1) if m else ""
            rows.append({"tipo":tipo_norm,"id":docid,"part":part})
    except Exception:
        pass

    # 3) Tesseract de respaldo full page
    if not rows:
        try:
            txt = page_text_tesseract(page)
            for ln in txt.splitlines():
                m = re.search(r"\b(CI|P)\b\s+([A-Za-z0-9\-]+).{0,120}?(\d{1,3}(?:[.,]\d+)?%?)\b", ln)
                if m:
                    rows.append({"tipo":m.group(1).upper(),"id":m.group(2).strip(),"part":m.group(3).strip()})
        except Exception:
            pass
    return rows

def extract_rows_from_page(page: fitz.Page):
    # 1) Tablas nativas
    try:
        tables = page.find_tables()
        if tables and tables.tables:
            tbl = max(tables.tables, key=lambda t: (t.bbox[2]-t.bbox[0])*(t.bbox[3]-t.bbox[1]))
            data = tbl.extract()
            start = 2 if len(data)>2 else 0
            rows=[]
            for r in data[start:]:
                tipo=(r[0] or "").strip().upper()
                doc_id=(r[1] or "").strip()
                part=(r[6] or "").strip() if len(r)>6 else ""
                if not any([tipo,doc_id,part]): continue
                rows.append({"tipo":tipo,"id":doc_id,"part":part})
            if rows:
                return rows
    except Exception:
        pass
    # 2) Palabras con coordenadas
    rows = extract_rows_by_words(page)
    if rows: return rows
    # 3) OCR IA (EasyOCR) + Tesseract
    rows = extract_rows_by_ocr(page)
    return rows

def status_row(pdf_name, rut_pj, page_num, msg):
    return {
        "Archivo": pdf_name,
        "Pagina": page_num if page_num is not None else "",
        "Rut_PJ": rut_pj,
        "Tipo_doc": "",
        "Rut_bf": "",
        "Rut_bf_num11": "",
        "porc_part_raw": "",
        "porc_part": "",
        "Estado": msg,
    }

def run_extraction_to_df(input_dir: Path):
    records=[]
    for pdf in sorted(input_dir.glob("*.pdf")):
        rut_pj = normalize_company_rut_from_filename(pdf.name)
        with fitz.open(pdf) as doc:
            best = select_best_page(doc)
            if best is None:
                records.append(status_row(pdf.name, rut_pj, None,
                    "Incidencia: no se encontró la sección 'Identificación de Beneficiarios Finales'."))
                continue
            page = doc.load_page(best)
            rows = extract_rows_from_page(page)
            if not rows:
                records.append(status_row(pdf.name, rut_pj, best+1,
                    "Incidencia: se encontró la sección pero no se pudo leer información de la tabla."))
                continue
            for row in rows:
                tipo = (row.get("tipo") or "").strip().upper()
                id_text = (row.get("id") or "").strip()
                p_val, p_raw = parse_percent_cell(row.get("part"))
                records.append({
                    "Archivo": pdf.name,
                    "Pagina": best + 1,
                    "Rut_PJ": rut_pj,                                     # 11 (con K si existe)
                    "Tipo_doc": "CI" if tipo == "CI" else ("P" if tipo == "P" else ""),
                    "Rut_bf": id_text,
                    "Rut_bf_num11": normalize_rut_11_keep_k(id_text) if tipo == "CI" else "",
                    "porc_part_raw": p_raw,
                    "porc_part": p_val,
                    "Estado": "Success",
                })
    df = pd.DataFrame(records, columns=[
        "Archivo","Pagina","Rut_PJ","Tipo_doc","Rut_bf","Rut_bf_num11","porc_part_raw","porc_part","Estado"
    ])
    return df

# === Ejecutar y visualizar (sin CSV) ===
input_dir = folder_pdf   # viene de la Parte 2
df = run_extraction_to_df(input_dir)
print(f"Filas obtenidas: {len(df)}")
df.head(30)


KeyboardInterrupt: 

creamos tabla local y tabla hitorica con la informacion de los pdf

In [ ]:
# === visualizacion de los datos extraidos ===
from datetime import datetime
import pandas as pd
from IPython.display import display, HTML

# df ya viene de run_extraction_to_csv(...)
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 1) Tabla de la corrida (solo visualización)
df_run = df.astype(str).copy()

# 2) Histórico en memoria (variable global 'hist_bf')
#    Si no existe, la creamos con mismas columnas que df_run
try:
    hist_bf
except NameError:
    hist_bf = pd.DataFrame(columns=df_run.columns)

# 3) Unimos histórico + nuevas filas y de-duplicamos (claves lógicas)
dedup_keys = ["Archivo", "Pagina", "Rut_PJ", "Tipo_doc", "Rut_bf", "porc_part_raw"]
hist_bf = pd.concat([hist_bf, df_run], ignore_index=True)
hist_bf.drop_duplicates(subset=dedup_keys, keep="last", inplace=True)

# 4) Fila marcador (al final) con la fecha/hora de esta actualización
marker = {col: "" for col in hist_bf.columns}
marker["Archivo"] = "data_date_part"
marker["Estado"]  = timestamp
hist_bf = pd.concat([hist_bf, pd.DataFrame([marker])], ignore_index=True)

# 5) Mostrar ambas tablas
display(HTML(f"<h3>Extracción (corrida): {timestamp}</h3>"))
display(df_run)

display(HTML("<h3>Histórico acumulado (en memoria)</h3>"))
display(hist_bf)
